<a href="https://colab.research.google.com/github/mohammadrahmani98/MachineLearning/blob/main/Raptor_Binder_Peptide_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Phase 1 Literature review**

This section of the Raptor notebook provides an automated tool to suggest relevant scientific publications for a specified target protein. By leveraging PubMed's extensive database, the tool identifies experimental studies that have discovered binding pockets, aiding in the design of binder peptides.

* You need to modify the last line of code of this part to write your protein target name to get your references,
* The suggested refrences can easily help to access the last relevant published paper which worked on the same targe.



In [ ]:
!pip install requests beautifulsoup4 spacy matplotlib seaborn xmltodict
!python -m spacy download en_core_web_sm

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import spacy
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import xmltodict

# NLP Processing
nlp = spacy.load("en_core_web_sm")

# Function to search PubMed using the API
def search_pubmed(protein_name, max_results=20):
    api_key = "4266b303fbe3a527d5d3f3d64b3ac2dac509"  # Replace with your PubMed API key
    search_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={protein_name}+binding+pocket+AND+experimental&retmax={max_results}&retmode=json&api_key={api_key}"
    response = requests.get(search_url)
    data = response.json()
    return data['esearchresult']['idlist']

# Function to fetch details of articles from PubMed using the API
def fetch_article_details(pubmed_ids):
    api_key = "4266b303fbe3a527d5d3f3d64b3ac2dac509"  # Replace with your PubMed API key
    ids = ",".join(pubmed_ids)
    fetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={ids}&retmode=xml&api_key={api_key}"
    response = requests.get(fetch_url)
    articles = xmltodict.parse(response.content)
    return articles['PubmedArticleSet']['PubmedArticle']

# Extract and Clean Text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Extract relevant information using NLP
def extract_protein_info(text):
    doc = nlp(text)
    proteins = defaultdict(list)
    for ent in doc.ents:
        if ent.label_ == "PROTEIN":
            proteins[ent.text].append(ent.sent.text)
    return proteins

# Filter for experimental studies mentioning binding pockets
def filter_relevant_articles(articles):
    relevant_articles = []
    for article in articles:
        try:
            abstract = article['MedlineCitation']['Article']['Abstract']['AbstractText']
            if 'binding pocket' in abstract and 'experimental' in abstract:
                relevant_articles.append(article)
        except:
            continue
    return relevant_articles[:8]  # Limit to top 8 relevant articles

# Main Function
def main(protein_name):
    # Step 1: Search PubMed for relevant articles
    pubmed_ids = search_pubmed(protein_name)

    # Step 2: Fetch details of the articles
    articles = fetch_article_details(pubmed_ids)

    # Step 3: Filter and select the most relevant articles
    relevant_articles = filter_relevant_articles(articles)

    # Step 4: Print references
    print("Suggested References:")
    for article in relevant_articles:
        title = article['MedlineCitation']['Article']['ArticleTitle']
        journal = article['MedlineCitation']['Article']['Journal']['Title']
        pub_date = article['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
        pub_year = pub_date.get('Year', 'Unknown Year')
        authors = article['MedlineCitation']['Article']['AuthorList']['Author']
        author_names = ", ".join([f"{author.get('LastName', '')} {author.get('Initials', '')}" for author in authors if 'LastName' in author and 'Initials' in author])
        print(f"Title: {title}\nJournal: {journal}\nPublication Year: {pub_year}\nAuthors: {author_names}\n")

# Example usage:
main("trypsin")


Suggested References:
Title: New insights on the binding of butyl-paraben to trypsin: experimental and computational approaches.
Journal: Journal of biomolecular structure & dynamics
Publication Year: 2023
Authors: Mostafavi ES, Asoodeh A, Chamani J

Title: Water Network in the Binding Pocket of Fluorinated BPTI-Trypsin Complexes─Insights from Simulation and Experiment.
Journal: The journal of physical chemistry. B
Publication Year: 2022
Authors: Wehrhan L, Leppkes J, Dimos N, Loll B, Koksch B, Keller BG

Title: Structural insights on the effects of mutation of a charged binding pocket residue on phosphopeptide binding to 14-3-3ζ protein.
Journal: Proteins
Publication Year: 2022
Authors: T S S, Dalvi S, Venkatraman P, Vemparala S

Title: Interactive molecular dynamics in virtual reality for accurate flexible protein-ligand docking.
Journal: PloS one
Publication Year: 2020
Authors: Deeks HM, Walters RK, Hare SR, O'Connor MB, Mulholland AJ, Glowacki DR

Title: Exploring the Ligand Bindin

#**Phase 2 -3 .RFdiffusion + ProtienMPNN**
RFdiffusion is a method for structure generation, with or without conditional information (a motif, target etc). It can perform a whole range of protein design challenges as we have outlined in the RFdiffusion

[![DOI](https://img.shields.io/badge/DOI-10.1101/zenodo.8246977-blue)](https://www.biorxiv.org/content/10.1101/2022.12.09.519842v2)


**Instructions**
---
---

Use `contigs` to define continious chains. Use a `:` to define multiple contigs and a `/` to define mutliple segments within a contig.
For example:

**unconditional**
- `contigs='100'` - diffuse **monomer** of length 100
- `contigs='50:100'` - diffuse **hetero-oligomer** of lengths 50 and 100
- `contigs='50'` `symmetry='cyclic'` `order=2` - make two copies of the defined contig(s) and add a symmetry constraint, for **homo-oligomeric** diffusion.

**binder design**
- `contigs='A:50'` `pdb='4N5T'` - diffuse a **binder** of length 50 to chain A of defined PDB.
- `contigs='E6-155:70-100'` `pdb='5KQV'` `hotspot='E64,E88,E96'` - diffuse a **binder** of length 70 to 100 (sampled randomly) to chain E and defined hotspot(s).

**motif scaffolding**
 - `contigs='40/A163-181/40'` `pdb='5TPN'`
 - `contigs='A3-30/36/A33-68'` `pdb='6MRR'` - diffuse a loop of length 36 between two segments of defined PDB ranges.

**partial diffusion**
- `contigs=''` `pdb='6MRR'` - noise all coordinates
- `contigs='A1-10'` `pdb='6MRR'` - keep first 10 positions fixed, noise the rest
- `contigs='A'` `pdb='1SSC'` - fix chain A, noise the rest

*hints and tips*
- `pdb=''` leave blank to get an upload prompt
- `contigs='50-100'` use dash to specify a range of lengths to sample from

In [ ]:
#@title setup **RFdiffusion** (~3min)
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
  os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
  # 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
  os.system("pip install --no-dependencies dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html")
  os.system("pip install --no-dependencies e3nn==0.3.3 opt_einsum_fx")
  os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt","Complex_beta_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    if not os.path.isfile(f"{pdb_code}.pdb1"):
      os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz")
      os.system(f"gunzip {pdb_code}.pdb1.gz")
    return f"{pdb_code}.pdb1"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

def run_ananas(pdb_str, path, sym=None):
  pdb_filename = f"outputs/{path}/ananas_input.pdb"
  out_filename = f"outputs/{path}/ananas.json"
  with open(pdb_filename,"w") as handle:
    handle.write(pdb_str)

  cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
  if sym is None: os.system(cmd)
  else: os.system(f"{cmd} {sym}")

  # parse results
  try:
    out = json.loads(open(out_filename,"r").read())
    results,AU = out[0], out[-1]["AU"]
    group = AU["group"]
    chains = AU["chain names"]
    rmsd = results["Average_RMSD"]
    print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

    C = np.array(results['transforms'][0]['CENTER'])
    A = [np.array(t["AXIS"]) for t in results['transforms']]

    # apply symmetry and filter to the asymmetric unit
    new_lines = []
    for line in pdb_str.split("\n"):
      if line.startswith("ATOM"):
        chain = line[21:22]
        if chain in chains:
          x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
          if group[0] == "c":
            x = sym_it(x,C,A[0])
          if group[0] == "d":
            x = sym_it(x,C,A[1],A[0])
          coord_str = "".join(["{:8.3f}".format(a) for a in x])
          new_lines.append(line[:30]+coord_str+line[54:])
      else:
        new_lines.append(line)
    return results, "\n".join(new_lines)

  except:
    return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

  def run_command_and_get_pid(command):
    pid_file = '/dev/shm/pid'
    os.system(f'nohup {command} & echo $! > {pid_file}')
    with open(pid_file, 'r') as f:
      pid = int(f.read().strip())
    os.remove(pid_file)
    return pid
  def is_process_running(pid):
    try:
      os.kill(pid, 0)
    except OSError:
      return False
    else:
      return True

  run_output = widgets.Output()
  progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
  display(widgets.VBox([progress, run_output]))

  # clear previous run
  for n in range(steps):
    if os.path.isfile(f"/dev/shm/{n}.pdb"):
      os.remove(f"/dev/shm/{n}.pdb")

  pid = run_command_and_get_pid(command)
  try:
    fail = False
    for _ in range(num_designs):

      # for each step check if output generated
      for n in range(steps):
        wait = True
        while wait and not fail:
          time.sleep(0.1)
          if os.path.isfile(f"/dev/shm/{n}.pdb"):
            pdb_str = open(f"/dev/shm/{n}.pdb").read()
            if pdb_str[-3:] == "TER":
              wait = False
            elif not is_process_running(pid):
              fail = True
          elif not is_process_running(pid):
            fail = True

        if fail:
          progress.bar_style = 'danger'
          progress.description = "failed"
          break

        else:
          progress.value = (n+1) / steps
          if visual != "none":
            with run_output:
              run_output.clear_output(wait=True)
              if visual == "image":
                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                fig = plt.figure()
                fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1)
                plt.show()
              if visual == "interactive":
                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                view.addModel(pdb_str,'pdb')
                view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                view.zoomTo()
                view.show()
        if os.path.exists(f"/dev/shm/{n}.pdb"):
          os.remove(f"/dev/shm/{n}.pdb")
      if fail:
        progress.bar_style = 'danger'
        progress.description = "failed"
        break

    while is_process_running(pid):
      time.sleep(0.1)

  except KeyboardInterrupt:
    os.kill(pid, signal.SIGTERM)
    progress.bar_style = 'danger'
    progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, partial_T="auto",
                  num_designs=1, use_beta_model=False, visual="none"):

  full_path = f"outputs/{path}"
  os.makedirs(full_path, exist_ok=True)
  opts = [f"inference.output_prefix={full_path}",
          f"inference.num_designs={num_designs}"]

  if chains == "": chains = None

  # determine symmetry type
  if symmetry in ["auto","cyclic","dihedral"]:
    if symmetry == "auto":
      sym, copies = None, 1
    else:
      sym, copies = {"cyclic":(f"c{order}",order),
                     "dihedral":(f"d{order}",order*2)}[symmetry]
  else:
    symmetry = None
    sym, copies = None, 1

  # determine mode
  contigs = contigs.replace(","," ").replace(":"," ").split()
  is_fixed, is_free = False, False
  fixed_chains = []
  for contig in contigs:
    for x in contig.split("/"):
      a = x.split("-")[0]
      if a[0].isalpha():
        is_fixed = True
        if a[0] not in fixed_chains:
          fixed_chains.append(a[0])
      if a.isnumeric():
        is_free = True
  if len(contigs) == 0 or not is_free:
    mode = "partial"
  elif is_fixed:
    mode = "fixed"
  else:
    mode = "free"

  # fix input contigs
  if mode in ["partial","fixed"]:
    pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
    if symmetry == "auto":
      a, pdb_str = run_ananas(pdb_str, path)
      if a is None:
        print(f'ERROR: no symmetry detected')
        symmetry = None
        sym, copies = None, 1
      else:
        if a["group"][0] == "c":
          symmetry = "cyclic"
          sym, copies = a["group"], int(a["group"][1:])
        elif a["group"][0] == "d":
          symmetry = "dihedral"
          sym, copies = a["group"], 2 * int(a["group"][1:])
        else:
          print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
          symmetry = None
          sym, copies = None, 1

    elif mode == "fixed":
      pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

    pdb_filename = f"{full_path}/input.pdb"
    with open(pdb_filename, "w") as handle:
      handle.write(pdb_str)

    parsed_pdb = parse_pdb(pdb_filename)
    opts.append(f"inference.input_pdb={pdb_filename}")
    if mode in ["partial"]:
      if partial_T == "auto":
        iterations = int(80 * (iterations / 200))
      else:
        iterations = int(partial_T)
      opts.append(f"diffuser.partial_T={iterations}")
      contigs = fix_partial_contigs(contigs, parsed_pdb)
    else:
      opts.append(f"diffuser.T={iterations}")
      contigs = fix_contigs(contigs, parsed_pdb)
  else:
    opts.append(f"diffuser.T={iterations}")
    parsed_pdb = None
    contigs = fix_contigs(contigs, parsed_pdb)

  if hotspot is not None and hotspot != "":
    hotspot = ",".join(hotspot.replace(","," ").split())
    opts.append(f"ppi.hotspot_res='[{hotspot}]'")

  # setup symmetry
  if sym is not None:
    sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
    if add_potential:
      sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                   "potentials.olig_intra_all=True","potentials.olig_inter_all=True",
                   "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
    opts = sym_opts + opts
    contigs = sum([contigs] * copies,[])

  opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
  opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
  if use_beta_model:
    opts += ["inference.ckpt_override_path=./RFdiffusion/models/Complex_beta_ckpt.pt"]

  print("mode:", mode)
  print("output:", full_path)
  print("contigs:", contigs)

  opts_str = " ".join(opts)
  cmd = f"./RFdiffusion/run_inference.py {opts_str}"
  print(cmd)

  # RUN
  run(cmd, iterations, num_designs, visual=visual)

  # fix pdbs
  for n in range(num_designs):
    pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb",
            f"outputs/traj/{path}_{n}_Xt-1_traj.pdb",
            f"{full_path}_{n}.pdb"]
    for pdb in pdbs:
      with open(pdb,"r") as handle: pdb_str = handle.read()
      with open(pdb,"w") as handle: handle.write(fix_pdb(pdb_str, contigs))

  return contigs, copies

In [ ]:
%%time
#@title run **RFdiffusion** to generate a backbone
name = "test" #@param {type:"string"}
contigs = "100" #@param {type:"string"}
pdb = "" #@param {type:"string"}
iterations = 50 #@param ["25", "50", "100", "150", "200"] {type:"raw"}
hotspot = "" #@param {type:"string"}
num_designs = 1 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
visual = "image" #@param ["none", "image", "interactive"]
#@markdown ---
#@markdown **symmetry** settings
#@markdown ---
symmetry = "none" #@param ["none", "auto", "cyclic", "dihedral"]
order = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"] {type:"raw"}
chains = "" #@param {type:"string"}
add_potential = True #@param {type:"boolean"}
#@markdown - `symmetry='auto'` enables automatic symmetry dectection with [AnAnaS](https://team.inria.fr/nano-d/software/ananas/).
#@markdown - `chains="A,B"` filter PDB input to these chains (may help auto-symm detector)
#@markdown - `add_potential` to discourage clashes between chains
#@markdown ---
#@markdown **advanced** settings
#@markdown ---
partial_T = "auto" # @param ["auto", "10", "20", "40", "60", "80"]
#@markdown - specify number of noising steps (only used for the partial diffusion protocol)
use_beta_model = False #@param {type:"boolean"}
#@markdown - if you are seeing lots of helices, switch to the "beta" params for a better SSE balance.

# determine where to save
path = name
while os.path.exists(f"outputs/{path}_0.pdb"):
  path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "use_beta_model":use_beta_model,
         "visual":visual,
         "partial_T":partial_T}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')

contigs, copies = run_diffusion(**flags)

In [ ]:
#@title Display 3D structure {run: "auto"}
animate = "none" #@param ["none", "movie", "interactive"]
color = "chain" #@param ["rainbow", "chain", "plddt"]
denoise = True
dpi = 100 #@param ["100", "200", "400"] {type:"raw"}
from colabdesign.shared.plot import pymol_color_list
from colabdesign.rf.utils import get_ca, get_Ls, make_animation
from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def plot_pdb(num=0):
  if denoise:
    pdb_traj = f"outputs/traj/{path}_{num}_pX0_traj.pdb"
  else:
    pdb_traj = f"outputs/traj/{path}_{num}_Xt-1_traj.pdb"
  if animate in ["none","interactive"]:
    hbondCutoff = 4.0
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if animate == "interactive":
      pdb_str = open(pdb_traj,'r').read()
      view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    else:
      pdb = f"outputs/{path}_{num}.pdb"
      pdb_str = open(pdb,'r').read()
      view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    if color == "rainbow":
      view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
      for n,chain,c in zip(range(len(contigs)),
                              alphabet_list,
                              pymol_color_list):
          view.setStyle({'chain':chain},{'cartoon': {'color':c}})
    else:
      view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
    view.zoomTo()
    if animate == "interactive":
      view.animate({'loop': 'backAndForth'})
    view.show()
  else:
    Ls = get_Ls(contigs)
    xyz, bfact = get_ca(pdb_traj, get_bfact=True)
    xyz = xyz.reshape((-1,sum(Ls),3))[::-1]
    bfact = bfact.reshape((-1,sum(Ls)))[::-1]
    if color == "chain":
      display(HTML(make_animation(xyz, Ls=Ls, dpi=dpi, ref=-1)))
    elif color == "rainbow":
      display(HTML(make_animation(xyz, dpi=dpi, ref=-1)))
    else:
      display(HTML(make_animation(xyz, plddt=bfact*100, dpi=dpi, ref=-1)))


if num_designs > 1:
  output = widgets.Output()
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
      options=[(f'{k}',k) for k in range(num_designs)],
      value=0, description='design:',
  )
  dropdown.observe(on_change)
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()

In [ ]:
%%time
#@title run **ProteinMPNN** to generate a sequence and **AlphaFold** to validate
#@markdown ProteinMPNN Settings
num_seqs = 8 #@param ["1", "2", "4", "8", "16", "32", "64"] {type:"raw"}
mpnn_sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
rm_aa = "C" #@param {type:"string"}
use_solubleMPNN = False #@param {type:"boolean"}
#@markdown - `mpnn_sampling_temp` - control diversity of sampled sequences. (higher = more diverse).
#@markdown - `rm_aa='C'` - do not use [C]ysteines.
#@markdown - `use_solubleMPNN` - use weights trained only on soluble proteins. See [preprint](https://www.biorxiv.org/content/10.1101/2023.05.09.540044v2).
#@markdown
#@markdown AlphaFold Settings
initial_guess = False #@param {type:"boolean"}
#@markdown - soft initialization with desired coordinates, see [paper](https://www.nature.com/articles/s41467-023-38328-5).
num_recycles = 1 #@param ["0", "1", "2", "3", "6", "12"] {type:"raw"}
#@markdown - for **binder** design, we recommend `initial_guess=True num_recycles=3`
use_multimer = False #@param {type:"boolean"}
#@markdown - `use_multimer` - use AlphaFold Multimer v3 params for prediction.

if not os.path.isfile("params/done.txt"):
  print("downloading AlphaFold params...")
  while not os.path.isfile("params/done.txt"):
    time.sleep(5)

contigs_str = ":".join(contigs)
opts = [f"--pdb=outputs/{path}_0.pdb",
        f"--loc=outputs/{path}",
        f"--contig={contigs_str}",
        f"--copies={copies}",
        f"--num_seqs={num_seqs}",
        f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}",
        f"--mpnn_sampling_temp={mpnn_sampling_temp}",
        f"--num_designs={num_designs}"]
if initial_guess: opts.append("--initial_guess")
if use_multimer: opts.append("--use_multimer")
if use_solubleMPNN: opts.append("--use_soluble")
opts = ' '.join(opts)
!python colabdesign/rf/designability_test.py {opts}

In [ ]:
#@title Display best result
import py3Dmol
def plot_pdb(num = "best"):
  if num == "best":
    with open(f"outputs/{path}/best.pdb","r") as f:
      # REMARK 001 design {m} N {n} RMSD {rmsd}
      info = f.readline().strip('\n').split()
    num = info[3]
  hbondCutoff = 4.0
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(f"outputs/{path}_{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  pdb_str = open(f"outputs/{path}/best_design{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})

  view.setStyle({"model":0},{'cartoon':{}}) #: {'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.setStyle({"model":1},{'cartoon':{'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.zoomTo()
  view.show()

if num_designs > 1:
  def on_change(change):
    if change['name'] == 'value':
      with output:
        output.clear_output(wait=True)
        plot_pdb(change['new'])
  dropdown = widgets.Dropdown(
    options=["best"] + [str(k) for k in range(num_designs)],
    value="best",
    description='design:',
  )
  dropdown.observe(on_change)
  output = widgets.Output()
  display(widgets.VBox([dropdown, output]))
  with output:
    plot_pdb(dropdown.value)
else:
  plot_pdb()

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive,
#@markdown try disabling your adblocker and run this cell again.
#@markdown  If that fails click on the little folder icon to the
#@markdown  left, navigate to file: `name.result.zip`,
#@markdown  right-click and select \"Download\"
#@markdown (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).
!zip -r {path}.result.zip outputs/{path}* outputs/traj/{path}*
files.download(f"{path}.result.zip")

#**Phase 4 .Autodock Vina**
 **Virtual Screening**

This labodock notebook is designed for conducting structure-based virtual screening procedures using **Autodock Vina 1.2.5** and performing binding interaction analysis with **PLIP 2.3.0**.

[![DOI](https://img.shields.io/badge/DOI-10.5281/zenodo.8246977-blue)](https://doi.org/10.5281/zenodo.8246977)


---
---
 **01 | Setting Up Environment**

Firstly, we install all the necessary libraries and packages.

The main packages that will be installed include:

+ AutoDock Vina 1.2.5 (https://vina.scripps.edu/)
+ Condacolab 0.1.7 (https://github.com/conda-incubator/condacolab)
+ Meeko 0.5.0 (https://github.com/forlilab/Meeko)
+ OpenBabel 3.1.1 (https://github.com/openbabel/openbabel)
+ PLIP 2.3.0 (https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index)
+ Py3Dmol 2.0.3 (https://pypi.org/project/py3Dmol/)
+ Rdkit 2022.9.5 (https://github.com/rdkit/rdkit)
+ Spyrmsd 0.6.0 (https://github.com/RMeli/spyrmsd)

All the above libraries and packages are currently supported by **Python 3.10**.

In [ ]:
# @title **Install packages and dependencies**
# @markdown Thanks to **`mamba`**, the installation takes **around 2 mins**. \
# @markdown It will **restart** the kernel (session).

import sys
import time
import contextlib

with open('/content/labodock_install.log', 'a') as inpt:
    with contextlib.redirect_stdout(inpt):

        # -- Start installation --
        start = time.time()
        !rm -r /content/sample_data
        !wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64 -O vina
        !wget https://raw.githubusercontent.com/RyanZR/labodock/develop/environment.yml -O environment.yml
        !chmod u+x vina environment.yml

        !pip install condacolab==0.1.8
        import condacolab
        condacolab.install_mambaforge()
        !mamba env update --file environment.yml
        end = time.time()
        # -- End installation --

        print(f'+ Time elapsed: ' + time.strftime('%Mm %Ss', time.gmtime(end - start)))

In [ ]:
# @title **Import Python modules**
# @markdown This will import the necessary Python modules.

# Internal Modules
import os
import glob
import time
import shutil

# External modules
import py3Dmol

# Data-related
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Google Colab-related
from google.colab import drive
from tqdm.notebook import tqdm

# Docking-related
from rdkit import Chem, RDLogger
from rdkit.Chem import rdFMCS, AllChem, Draw
from spyrmsd import io, rmsd

# Binding interaction-related
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex

print(f'+ Import completed')

In [ ]:
# @title **Create folders**
# @markdown Enter a **< Job Name >** to create a working directory containing:
# @markdown + **`PROTEIN`** : Stores the target protein
# @markdown + **`LIGAND`** : Stores the ligand to be docked
# @markdown + **`NATIVE`** : Stores the native ligand of target protein
# @markdown + **`DOCK`** : Stores the docking pose and scoring data
# @markdown + **`INTERACTION`** : Stores the binding interaction data

Job_name = '7N3N' # @param {type: 'string'}

invalid_chars = '^<>/\{}[]~`$ '
assert Job_name, 'Do not leave this blank.'
assert not set(invalid_chars).intersection(Job_name), 'Disallowed characters.'

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, 'PROTEIN')
LIG_FLD = os.path.join(WRK_DIR, 'LIGAND')
NTV_FLD = os.path.join(WRK_DIR, 'NATIVE')
DCK_FLD = os.path.join(WRK_DIR, 'DOCKING')
INT_FLD = os.path.join(WRK_DIR, 'INTERACTION')

folders = [WRK_DIR, PRT_FLD, LIG_FLD, NTV_FLD, DCK_FLD, INT_FLD]

for folder in folders:
    if os.path.exists(folder):
        print(f'+ Folder existed: {folder}')
    else:
        os.mkdir(folder)
        print(f'+ Folder created: {folder}')

In [ ]:
# @title **Set up utilities**
# @markdown This creates important custom functions and methods for later
# @markdown docking and binding interaction study.

%alias vina ./vina

#############################################
# Suppress Warnings

RDLogger.DisableLog('rdApp.warning')

#############################################
# Grid Box Calculation Methods

class GridBox:

    ranges = tuple[list[float], list[float], list[float]]
    coords = tuple[float, float, float]
    center_bxsize = tuple[tuple[float, float, float], tuple[float, float, float]]

    def __init__(self, inpt_file: str) -> None:
        self.inpt = open(inpt_file, 'r')
        self.data = self.inpt.read()
        self.cmol = Chem.MolFromPDBBlock(self.data)
        self.conf = self.cmol.GetConformer()
        self.ntom = self.cmol.GetNumAtoms()
        self.inpt.close()

    def update_gridbox(self, mol_block: str) -> None:
        self.cmol = Chem.MolFromPDBBlock(mol_block)
        self.conf = self.cmol.GetConformer()
        self.ntom = self.cmol.GetNumAtoms()

    def compute_coords(self) -> ranges:
        x_coord = [self.conf.GetAtomPosition(c).x for c in range(self.ntom)]
        y_coord = [self.conf.GetAtomPosition(c).y for c in range(self.ntom)]
        z_coord = [self.conf.GetAtomPosition(c).z for c in range(self.ntom)]
        return x_coord, y_coord, z_coord

    def compute_ranges(self) -> ranges:
        x, y, z = self.compute_coords()
        x_range = [min(x), max(x)]
        y_range = [min(y), max(y)]
        z_range = [min(z), max(z)]
        return x_range, y_range, z_range

    def compute_center(self, use_range: bool = True) -> coords:
        x, y, z = self.compute_ranges() if use_range else self.compute_coords()
        x_center = round(np.mean(x), 3)
        y_center = round(np.mean(y), 3)
        z_center = round(np.mean(z), 3)
        return x_center, y_center, z_center

    def generate_res_molblock(self, residues_list: list[str]) -> str:
        res_lines = [line for line in self.data.split('\n')
                     if line[22:26].lstrip() in residues_list
                     and 'END' not in line]
        res_block = '\n'.join(res_lines)
        return res_block

    def labox(self, scale: float = 2.0) -> coords:
        xr, yr, zr = self.compute_ranges()
        center = self.compute_center()
        bxsize = (round(abs(xr[0] - xr[1]) * scale, 3),
                  round(abs(yr[0] - yr[1]) * scale, 3),
                  round(abs(zr[0] - zr[1]) * scale, 3))
        return center, bxsize

    def eboxsize(self, gy_box_ratio: float = 0.23, modified: bool = False) -> center_bxsize:
        xc, yc, zc = self.compute_coords()
        center = self.compute_center(modified)
        distsq = [(x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2
                  for x, y, z in zip(xc, yc, zc)]
        bxsize = (round(np.sqrt(sum(distsq) / len(xc)) / gy_box_ratio, 3),) * 3
        return center, bxsize

    def autodock_grid(self) -> center_bxsize:
        xr, yr, zr = self.compute_ranges()
        center = self.compute_center()
        bxsize = (22.5, 22.5, 22.5)
        return center, bxsize

    def defined_by_res(self, residue_number: str, scale: float = 1.25) -> center_bxsize:
        res_list = residue_number.replace(',', ' ').split()
        res_block = self.generate_res_molblock(res_list)
        self.update_gridbox(res_block)
        return self.labox(scale=scale)

#############################################
# RMSD Calculation Methods

class ComputeRMSD:

    def __init__(self) -> None:
        self.MCS_mol = None
        self.MCS_png = None

    def load_molecule(self, inpt_file: str, remove_Hs: bool = True) -> tuple:
        molecule = io.loadmol(inpt_file)
        molecule.strip() if remove_Hs else None
        name = os.path.basename(inpt_file).split('.')[0]
        coor = molecule.coordinates
        anum = molecule.atomicnums
        mtrx = molecule.adjacency_matrix
        cmol = Chem.MolFromPDBFile(inpt_file)
        return name, coor, anum, mtrx, cmol

    def mol_to_png(self, mol: object) -> object:
        legend = 'Maximum Common Substructure'
        png = Draw.MolToImage(mol, legend=legend)
        return png

    def find_MCS(self, ref: tuple, lig: tuple) -> object:
        if self.MCS_mol is None:
            MCS_obj = rdFMCS.FindMCS([ref[4], lig[4]])
            MCS_mol = Chem.MolFromSmarts(MCS_obj.smartsString)
            MCS_png = self.mol_to_png(MCS_mol)
            self.MCS_mol = MCS_mol
            self.MCS_png = MCS_png
        return self.MCS_mol

    def hung_RMSD(self, ref: tuple, lig: tuple) -> float:
        try:
            hRMSD = round(rmsd.hrmsd(ref[1], lig[1], ref[2], lig[2]), 3)
        except:
            hRMSD = 'ERROR'
        return hRMSD

    def symm_RMSD(self, ref: tuple, lig: tuple, minimise: bool = False) -> float:
        try:
            sRMSD = round(rmsd.symmrmsd(ref[1], lig[1], ref[2], lig[2], ref[3], lig[3], minimize=minimise), 3)
        except:
            sRMSD = 'ERROR'
        return sRMSD

    def labo_RMSD(self, ref: tuple, lig: tuple) -> float:
        mol_substr = self.find_MCS(ref, lig)
        ref_substr = ref[4].GetSubstructMatch(mol_substr)
        lig_substr = lig[4].GetSubstructMatch(mol_substr)

        distsq = []
        for ref_atom, lig_atom in zip(ref_substr, lig_substr):
            ref_pos = ref[4].GetConformer().GetAtomPosition(ref_atom)
            lig_pos = lig[4].GetConformer().GetAtomPosition(lig_atom)
            ref_coord = np.array((ref_pos.x, ref_pos.y, ref_pos.z))
            lig_coord = np.array((lig_pos.x, lig_pos.y, lig_pos.z))
            coo_dist = np.linalg.norm(ref_coord - lig_coord)
            distsq.append(coo_dist ** 2)

        try:
            lRMSD = round(np.sqrt(sum(distsq)/len(distsq)), 3)
        except:
            lRMSD = 'ERROR'
        return lRMSD

    def rmsd_report(self,
                    ref: tuple,
                    lig: tuple,
                    lRMSD: bool = True,
                    hRMSD: bool = True,
                    sRMSD: bool = True
                    ) -> dict[str: list[float]]:
        report = {}
        report['NAME'] = [lig[0]]
        report['LABO_RMSD'] = [self.labo_RMSD(ref, lig)] if lRMSD else None
        report['HUNG_RMSD'] = [self.hung_RMSD(ref, lig)] if hRMSD else None
        report['SYMM_RMSD'] = [self.symm_RMSD(ref, lig)] if sRMSD else None
        report = {k: v for k, v in report.items() if v is not None}
        return report

#############################################
# AA Consntant and Bond Colour Dictionary

# Kyte and Doolittle Hydropathy Scale (1982)
AA_HB = {'ALA':  1.8, 'ARG': -4.5, 'ASN': -3.5, 'ASP': -3.5, 'CYS':  2.5,
         'GLN': -3.5, 'GLU': -3.5, 'GLY': -0.4, 'HIS': -3.2, 'ILE':  4.5,
         'LEU':  3.8, 'LYS': -3.9, 'MET':  1.9, 'PHE':  2.8, 'PRO': -1.6,
         'SER': -0.8, 'THR': -0.7, 'TRP': -0.9, 'TYR': -1.3, 'VAL':  4.2}

# University of Calgary PI Scale
AA_PI = {'ALA':  6.0, 'ARG': 10.76, 'ASN': 5.41, 'ASP': 2.77, 'CYS': 5.07,
         'GLN': 5.65, 'GLU':  3.22, 'GLY': 5.97, 'HIS': 7.59, 'ILE': 6.02,
         'LEU': 5.98, 'LYS':  9.74, 'MET': 5.74, 'PHE': 5.48, 'PRO':  6.3,
         'SEC': 5.68, 'SER':  5.68, 'THR':  5.6, 'TRP': 5.89, 'TYR': 5.66,
         'VAL': 5.96}

BOND_COL = {'HYDROPHOBIC': ['0x59e382', 'GREEN'],
            'HBOND': ['0x59bee3', 'LIGHT BLUE'],
            'WATERBRIDGE': ['0x4c4cff', 'BLUE'],
            'SALTBRIDGE': ['0xefd033', 'YELLOW'],
            'PISTACKING': ['0xb559e3', 'PURPLE'],
            'PICATION': ['0xe359d8', 'VIOLET'],
            'HALOGEN': ['0x59bee3', 'LIGHT BLUE'],
            'METAL':['0xe35959', 'ORANGE']}

#############################################
# AA-to-Colour Converter Function

def sequential_gradient(value: float,
                        min_value: float,
                        max_value: float,
                        targ_colour: str = '00ff00',
                        interpolation: float = 0.0
                        ) -> str:
    norm_val = (value - min_value) / (max_value - min_value)

    rgb = tuple(int(targ_colour[d:d+2], 16) for d in (0, 2, 4))
    r = int(255 - (255 - rgb[0]) * (1 - interpolation) * norm_val)
    g = int(255 - (255 - rgb[1]) * (1 - interpolation) * norm_val)
    b = int(255 - (255 - rgb[2]) * (1 - interpolation) * norm_val)

    hex_code = f'#{r:02x}{g:02x}{b:02x}'
    return hex_code

def diverging_gradient(value: float,
                       min_value: float,
                       max_value: float,
                       base_colour: str = 'ff0000',
                       targ_colour: str = '0000ff',
                       interpolation: float = 0.3
                       ) -> str:
    norm_val = (value - min_value) / (max_value - min_value)

    white = (255, 255, 255)
    rgb_A = tuple(int(base_colour[d:d+2], 16) for d in (0, 2, 4))
    rgb_B = tuple(int(targ_colour[d:d+2], 16) for d in (0, 2, 4))

    if norm_val < 0.5 - interpolation / 2:
        factor = norm_val / (0.5 - interpolation / 2)
        r = int(rgb_A[0] + (white[0] - rgb_A[0]) * factor)
        g = int(rgb_A[1] + (white[1] - rgb_A[1]) * factor)
        b = int(rgb_A[2] + (white[2] - rgb_A[2]) * factor)
    elif norm_val > 0.5 + interpolation / 2:
        factor = (norm_val - 0.5 - interpolation / 2) / (0.5 - interpolation / 2)
        r = int(white[0] + (rgb_B[0] - white[0]) * factor)
        g = int(white[1] + (rgb_B[1] - white[1]) * factor)
        b = int(white[2] + (rgb_B[2] - white[2]) * factor)
    else:
        r, g, b = white

    hex_code = f'#{r:02x}{g:02x}{b:02x}'
    return hex_code

def a2c_converter(aa_map: dict, grad_func: 'function') -> dict:
    min_value = min(aa_map.values())
    max_value = max(aa_map.values())
    aa_dict = {aa: grad_func(value, min_value, max_value)
               for aa, value in aa_map.items()}
    return aa_dict

#############################################
# Built-in Styling Function

def builtin_style(style: str, opacity: float = 1.0) -> dict:
    match style:
        case _ if any(kw in style for kw in ('Carbon', 'chain', 'ssJmol', 'ssPyMol')):
            style_dict = {'colorscheme': style}
        case 'hydrophobicity':
            style_dict = {'colorscheme': {
                'prop': 'resn', 'map': a2c_converter(AA_HB, sequential_gradient)}}
        case 'isoelectric points':
            style_dict = {'colorscheme': {
                'prop': 'resn', 'map': a2c_converter(AA_PI, diverging_gradient)}}
        case 'b factor':
            style_dict = {'colorscheme': {
                'prop': 'b', 'gradient': 'rwb', 'min': 90, 'max': 50}}
        case _:
            style_dict = {'color': style}

    style_dict.update({'opacity': opacity, 'singleBonds': False})
    return style_dict

#############################################
# Built-in Colour Scale Function

def colour_scale(aa_map: dict, grad_func: 'function') -> None:
    min_value = min(aa_map.values())
    max_value = max(aa_map.values())

    linear_values = np.linspace(min_value, max_value, 100)
    colours = [grad_func(value, min_value, max_value)
               for value in linear_values]

    fig, ax = plt.subplots(figsize=(4.85, 0.25))
    norm_value = plt.Normalize(min_value, max_value)
    colour_map = plt.cm.colors.ListedColormap(colours)
    scalar_map = plt.cm.ScalarMappable(norm_value, colour_map)
    scalar_map.set_array([])

    cscale = plt.colorbar(scalar_map, ax, orientation='horizontal')
    cscale.set_ticks([min_value, max_value])

def show_cscale(rept_info: dict, surf_info: dict) -> None:

    def cs_selector() -> str:
        if any(surf_info):
            style = [*surf_info.values()][0]
        elif any(rept_info):
            style = [*rept_info.values()][0]
        else:
            style = None
        return style

    def cs_display(style: str):
        if style == 'hydrophobicity':
            label_title(style, 'Less', 'More')
            colour_scale(AA_HB, sequential_gradient)
        elif style == 'isoelectric points':
            label_title(style, 'Acid', 'Base')
            colour_scale(AA_PI, diverging_gradient)
        else:
            pass

    def label_title(text: str, min: str, max: str) -> None:
        print(f'-' * 55)
        print(f'{min}{text.upper():^47}{max}')
        print(f'-' * 55)

    cs_display(cs_selector())

#############################################
# Other Functions

def extract_config(inpt_file: str) -> tuple:
    with open(inpt_file, 'r') as inpt:
        data = [line.split() for line in inpt.readlines()]
    center = (float(data[0][2]), float(data[1][2]), float(data[2][2]))
    bxsize = (float(data[4][2]), float(data[5][2]), float(data[6][2]))
    return center, bxsize

def interaction_dict(inpt_file: str, interactions: str = '', usage: str = 'view' or 'lbsp') -> dict:

    usg_map = {'lbsp': 0, 'view': 1}

    def filter_df(int_df: pd.DataFrame, interactions: list = []) -> pd.DataFrame:
        int_df = int_df[int_df['BOND'].isin(interactions)] if interactions else int_df
        return int_df

    def s2f_dict(item: dict) -> dict:
        return {key: tuple(float(val) for val in value[1:-1].split(','))
                for key, value in item.items()}

    def b2c_dict(item: dict) -> dict:
        return {key: BOND_COL[val][usg_map[usage]] for key, val in item.items()}

    intrxn = interactions.replace(',', ' ').split()
    inter_df = pd.read_csv(inpt_file)
    int_dict = filter_df(inter_df, intrxn).to_dict()
    int_dict['LIGCOO'] = s2f_dict(int_dict['LIGCOO'])
    int_dict['PROTCOO'] = s2f_dict(int_dict['PROTCOO'])
    int_dict['COLOR'] = b2c_dict(int_dict['BOND'])

    return int_dict

def find_midpoint(coords: list) -> tuple[float, float, float]:
    return tuple(round(coord, 3) for coord in np.mean(coords, axis=0))

#############################################
# LaboSpace Viewer

class LaboSpace:

    residue_style = {
        'stick':
         {'colorscheme': 'orangeCarbon', 'radius': 0.15}}
    residue_label = {
        'alignment': 'bottomLeft',
        'showBackground': False,
        'inFront': True,
        'fontSize': 14,
        'fontColor': '0x000000',
        'screenOffset': {'x': 25, 'y': 25}}
    atom_label = {
        'alignment': 'bottomLeft',
        'showBackground': False,
        'inFront': True,
        'fontSize': 14,
        'fontColor': '0x000000',
        'screenOffset': {'x': 10, 'y': 10}}

    def __init__(self, vw: int = 500, vh: int = 500) -> None:
        self.mview = py3Dmol.view(width=vw, height=vh)
        self.count = -1
        self.residues = []

    def read_moldata(self, inpt_file: str) -> str:
        inpt = open(inpt_file, 'r')
        data = inpt.read()
        inpt.close()
        return data

    def load_receptor(self, inpt_file: str) -> object:
        data = self.read_moldata(inpt_file)
        self.mview.addModel(data, 'pdb')
        self.count += 1
        return self

    def load_ligand(self, inpt_file: str) -> object:
        data = self.read_moldata(inpt_file)
        self.mview.addModel(data)
        self.count += 1
        return self

    def set_style(self,
                  show_represent: bool = True,
                  represent_type: str = 'cartoon',
                  represent_style: dict = {}
                  ) -> object:
        if show_represent:
            self.mview.setStyle(
                {'model': self.count},
                {represent_type: represent_style})
        else:
            self.mview.setStyle(
                {'model': self.count},
                {})
        return self

    def add_style(self,
                  show_represent: bool = True,
                  represent_style: dict = {}
                  ) -> object:
        if show_represent:
            self.mview.addStyle(
                {'model': self.count},
                represent_style)
        return self

    def add_residues(self,
                     show_residues: bool = True,
                     residue_number: str = ''
                     ) -> object:
        if show_residues and residue_number:
            res = residue_number.replace(',', ' ').split()
            self.residues.extend(list(set(res)))
            self.mview.addStyle(
                {'and': [{'model': self.count}, {'resi': self.residues}]},
                self.residue_style)
            self.mview.addResLabels(
                {'and': [{'model': self.count}, {'resi': self.residues}]},
                self.residue_label)
        return self

    def add_surface(self,
                    show_surface: bool = True,
                    surface_type: str = 'SES',
                    surface_style: dict = {}
                    ) -> object:
        if show_surface:
            self.mview.addSurface(
                surface_type,
                surface_style,
                {'model': self.count})
        return self

    def add_gridbox(self,
                    show_gridbox: bool,
                    center: list[float],
                    bxsize: list[float]
                    ) -> object:
        if show_gridbox:
            bxi, byi, bzi = center
            bxf, byf, bzf = bxsize
            self.mview.addBox({
                'center': {'x': bxi, 'y': byi, 'z': bzi},
                'dimensions': {'w': bxf, 'h': byf, 'd': bzf},
                'color': 'skyBlue',
                'opacity': 0.6})
            self.mview.addLabel(
                f'center: {bxi:>8}, {byi:>8}, {bzi:>8}',
                {'showBackground': False,
                 'fontSize': 14,
                 'fontColor': '0x000000',
                 'useScreen': True,
                 'screenOffset': {'x': 15, 'y': 0}})
            self.mview.addLabel(
                f'bxsize: {bxf:>8}, {byf:>8}, {bzf:>8}',
                {'showBackground': False,
                 'fontSize': 14,
                 'fontColor': '0x000000',
                 'useScreen': True,
                 'screenOffset': {'x': 15, 'y': -20}})
        return self

    def add_interaction(self,
                        interaction_file: str,
                        show_interaction: bool = True,
                        select_interaction: list = []
                        ) -> object:
        if show_interaction:
            int_dict = interaction_dict(interaction_file, select_interaction, 'lbsp')
            dist = int_dict['DIST'].values()
            bond = int_dict['BOND'].values()
            resn = int_dict['RESNR'].values()
            ligcoo = int_dict['LIGCOO'].values()
            prtcoo = int_dict['PROTCOO'].values()
            color = int_dict['COLOR'].values()

            int_res = list(set(resn) - set(self.residues))
            self.residues.extend(int_res)
            self.mview.addStyle(
                {'and': [{'model': 0}, {'resi': int_res}]},
                self.residue_style)
            self.mview.addResLabels(
                {'and': [{'model': 0}, {'resi': int_res}]},
                self.residue_label)

            for dis, col, lig, prt in zip(dist, color, ligcoo, prtcoo):
                mid = find_midpoint([lig, prt])
                self.mview.addCylinder(
                    {'start': {'x': lig[0], 'y': lig[1], 'z': lig[2]},
                     'end': {'x': prt[0], 'y': prt[1], 'z': prt[2]},
                     'radius': 0.05,
                     'fromCap': 1,
                     'toCap': 1,
                     'color': col,
                     'dashed': True})
                self.mview.addLabel(
                    str(dis) + ' Å',
                    {'position': {'x': mid[0], 'y': mid[1], 'z': mid[2]},
                     'alignment': 'bottomLeft',
                     'inFront': False,
                     'backgroundColor': col,
                     'fontSize': 10,
                     'screenOffset': {'x': 10, 'y': 10}})
        return self

    def label_atoms(self, show_label: bool = False) -> object:
        # WARNING: Avoid applying on protein !!!
        if show_label:
            self.mview.addPropertyLabels(
                'atom',
                {'model': self.count},
                self.atom_label)
        return self

    def view_space(self,
                   zoom_model: int = -1,
                   slab_view: bool = False,
                   slab_model: int = -1,
                   background_colour: str = '0xFFFFFF'
                   ) -> None:
        self.mview.setBackgroundColor(background_colour)
        self.mview.setProjection('orthographic')
        self.mview.zoomTo({'model': zoom_model})
        self.mview.fitSlab({'model': slab_model}) if slab_view else None
        self.mview.show()

print(f'+ Methods and functions successfully built')

---
---
 **02 | Preparing the Receptor**

In this step, we lay the groundwork for virtual screening by preparing the protein of interest. We will obtain the protein's crystal structure in **`pdb`** format from the [RCSB Protein Data Bank (PDB)](https://www.rcsb.org). Then, we clean the structure by extraction method and seperate its existing subunits into individual files. Finally, we parameterise our target strucutre to generate the **`pdbqt`** file required for docking.

In [ ]:
# @title **Download protein PDB file**
# @markdown Enter the **< Accession ID >** to download the protein of interest.

Accession_ID = '7N3N' # @param {type: 'string'}

PDB_pdb = Accession_ID.upper() + '.pdb'
PDB_pdb_pFile = os.path.join(PRT_FLD, PDB_pdb)
Accession_ID_url = 'http://files.rcsb.org/download/' + PDB_pdb

!wget {Accession_ID_url} -O {PDB_pdb_pFile} -q
print(f'+ PDB downloaded: {PDB_pdb} > PROTEIN folder')
print(f'+ RCSB PDB link: https://www.rcsb.org/structure/{Accession_ID}')

In [ ]:
# @title **Clean protein structure and extract subunits**
# @markdown This generates a **clean** protein structure containing all the subunit(s). \
# @markdown The subunits are then extracted into individual **`pdb`** files. \
# @markdown *Note: `Subunits` and `Chains` are interchangeable to describe the
# @markdown multiple polypeptides that made up the protein.*

def oupt_parse(inpt_file: str) -> tuple:
    name = os.path.basename(inpt_file).split('.')[0]
    path = os.path.dirname(inpt_file)
    dirn = os.path.basename(path)
    return name, dirn, path

def ter_line(asn: str, resn: str, chid: str, resi: str) -> str:
    return f'TER   {asn}      {resn} {chid}{resi:>4} {" "*54}\n'

def extract_protein(inpt_file: str, oupt_file: str) -> None:
    oupt_name, oupt_dirn, oupt_path = oupt_parse(oupt_file)
    with open(inpt_file, 'r') as inpt, \
         open(oupt_file , 'w') as oupt:
        for line in inpt:
            record = line.split()[0]
            if 'ATOM' in record:
                oupt.write(line)
    print(f'+ Protein extracted: {oupt_name}.pdb > {oupt_dirn} folder')

def extract_chains(inpt_file: str) -> None:
    inpt_name, oupt_dirn, oupt_path = oupt_parse(inpt_file)
    with open(inpt_file, 'r') as inpt:
        data = inpt.readlines()

    chid = sorted(set(line[21:22] for line in data))
    chid_leng = len(chid)
    chid_list = ', '.join(chid)
    print(f'+ Chains detected: {chid_leng} ({chid_list})')

    for id in chid:
        oupt_name = inpt_name + '_' + id
        oupt_file = os.path.join(oupt_path, oupt_name + '.pdb')
        with open(oupt_file, 'w') as oupt:
            for line in data:
                if line[21:22] in id:
                    oupt.write(line)
                    asn = f'{int(line[6:11])+1:>5}'
                    resn = line[17:20]
                    resi = line[22:26]
            oupt.write(ter_line(asn, resn, id, resi))
            oupt.write('END')
        print(f'+ Chain extracted: {oupt_name}.pdb > {oupt_dirn} folder')

PDB_prot = Accession_ID + '_prot'
PDB_prot_pdb = PDB_prot + '.pdb'
PDB_prot_pdb_pFile = os.path.join(PRT_FLD, PDB_prot_pdb)

extract_protein(PDB_pdb_pFile, PDB_prot_pdb_pFile)
extract_chains(PDB_prot_pdb_pFile)

In [ ]:
# @title **Visualise 3D protein structure**
# @markdown Enter the **< Protein >** to be viewed.

View = '7N3N_prot_A' # @param {type: 'string'}
Representation_type = 'cartoon' # @param ['cartoon', 'cross', 'line', 'sphere', 'stick']
Representation_style = 'white' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Representation_opacity = 1 # @param {type:'slider', min:0, max:1, step:0.1}
Residue_number = '' # @param {type:'string'}
Surface_type = 'SES' # @param ['VDW', 'SAS', 'SES', 'MS']
Surface_style = 'hydrophobicity' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Surface_opacity = 1 # @param {type:'slider', min:0, max:1, step:0.1}
Show_representation = True # @param {type:'boolean'}
Show_residue = False # @param {type:'boolean'}
Show_surface = False # @param {type:'boolean'}

PROT_view_pFile = os.path.join(PRT_FLD, View + '.pdb')

LBSP = LaboSpace(960, 640)
LBSP.load_receptor(PROT_view_pFile)\
    .set_style(
        show_represent=Show_representation,
        represent_type=Representation_type,
        represent_style=builtin_style(
            style=Representation_style,
            opacity=Representation_opacity))\
    .add_residues(
        show_residues=Show_residue,
        residue_number=Residue_number)\
    .add_surface(
        show_surface=Show_surface,
        surface_type=Surface_type,
        surface_style=builtin_style(
            style=Surface_style,
            opacity=Surface_opacity))
LBSP.view_space()

show_cscale(
    {Show_representation: Representation_style},
    {Show_surface: Surface_style})

In [ ]:
# @title **Parameterise protein with Gasteiger charges**
# @markdown Enter the **< Target Protein >** to be parameterised.\
# @markdown This generate a **`protein.pdbqt`** file after:
# @markdown + Addition of Gasteiger Partial Charge
# @markdown + Addition of polar hydrogens
# @markdown + Removal of non-polar hydrogens

Target_protein = '7N3N_prot_A' # @param {type:'string'}

PROT_pdb = Target_protein + '.pdb'
PROT_pdbqt = Target_protein + '.pdbqt'
PROT_pdb_pFile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_dFile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_dFile = os.path.join(DCK_FLD, PROT_pdbqt)

!obabel {PROT_pdb_pFile} -xr -O {PROT_pdbqt_dFile} -h --partialcharge gasteiger > /dev/null 2>&1
!mk_prepare_receptor.py --pdbqt {PROT_pdbqt_dFile} -o {PROT_pdbqt_dFile} --skip_gpf > /dev/null 2>&1
print(f'+ Parameterisation: {PROT_pdb} > {PROT_pdbqt}')

shutil.copy(PROT_pdb_pFile, PROT_pdb_dFile)
print(f'+ {PROT_pdbqt} > DOCKING folder')
print(f'+ {PROT_pdb} > DOCKING folder')

---
---
 **03 | Preparing the Native Ligand** (optional)

Next, we retrieve the native ligand from the **`pdb`** file to serve as a reference for later comparison. Noted that **this step is optional** as not all **`pdb`** files contain native ligand. However, we highly recommend proceeding with this step for an insightful visual (3D inspection) and qualtitative (RMSD) comparison of binding pose.

In [ ]:
# @title **Retrieve native ligand and extract subunits**
# @markdown Enter the **< Keyword >** assigned for query \
# @markdown This generates a **clean** native ligand structure containing multiple
# @markdown binding poses corresponding to each protein subunit(s). \
# @markdown The binding poses are then extracted and corrected into individual
# @markdown **`pdb`** files.

Keyword = 'FUN' # @param {type: 'string'}

def extract_ligand(inpt_file: str, oupt_file: str, keyword: str) -> None:
    oupt_name, oupt_dirn, oupt_path = oupt_parse(oupt_file)
    with open(inpt_file, 'r') as inpt, \
         open(oupt_file, 'w') as oupt:
        for line in inpt:
            record = line.split()[0]
            lig_id = line[16:20].strip()
            if 'HETATM' in record and keyword in lig_id:
                assert len(lig_id) <= 3 or lig_id == keyword, f'Try keyword \'{lig_id}\''
                oupt.write(line)
    print(f'+ Ligand extracted: {oupt_name}.pdb > {oupt_dirn} folder')

def get_molblock(keyword: str) -> str:
    url_path = 'http://files.rcsb.org/ligands/download/' + keyword + '_model.sdf'
    sdf_file = os.path.join(NTV_FLD, keyword + '.sdf')
    os.system(f'wget {url_path} -O {sdf_file} -q')
    molblock = [mol for mol in  Chem.SDMolSupplier(sdf_file) if mol is not None][0]
    os.remove(sdf_file)
    return molblock

def correct_bond_order(inpt_list: list, temp: Chem.rdchem.Mol) -> None:
    for inpt_file in inpt_list:
        targ = AllChem.MolFromPDBFile(inpt_file)
        cmol = AllChem.AssignBondOrdersFromTemplate(temp, targ)
        pdbb = Chem.MolToPDBBlock(cmol, flavor=4)
        with open(inpt_file, 'w') as oupt:
            oupt.write(pdbb)

true_keyword = Keyword[-3:] if len(Keyword) > 3 else Keyword
print(f'+ RCSB PDB link: https://www.rcsb.org/ligand/{true_keyword}')

ntv_pdb = Keyword.upper() + '.pdb'
ntv_pdb_nFile = os.path.join(NTV_FLD, ntv_pdb)
extract_ligand(PDB_pdb_pFile, ntv_pdb_nFile, Keyword)
extract_chains(ntv_pdb_nFile)

ntv_nFiles = sorted(glob.glob(NTV_FLD + '/' + Keyword + '_*.pdb'))
ntv_smiles = get_molblock(true_keyword)
correct_bond_order(ntv_nFiles, ntv_smiles)

In [ ]:
# @title **Visualise 3D native ligand**
# @markdown Enter the **< Native Ligand >** to be viewed.

View = 'FUN_A' # @param {type:'string'}
Representation_type = 'stick' # @param ['line', 'sphere', 'stick']
Show_atom_labels = False # @param {type:'boolean'}

NTV_view_nFile = os.path.join(NTV_FLD, View + '.pdb')

LBSP = LaboSpace(960, 640)
LBSP.load_ligand(NTV_view_nFile)\
    .set_style(
        show_represent=True,
        represent_type=Representation_type,
        represent_style={'colorscheme': 'lightGreyCarbon'})\
    .label_atoms(
        show_label=Show_atom_labels)
LBSP.view_space()

In [ ]:
# @title **Select native ligand**
# @markdown Enter the **< Target Native Ligand >** for later comparison.

Target_native_ligand = 'FUN_A' # @param {type:'string'}

NTV_pdb = Target_native_ligand + '.pdb'
NTV_pdb_nFile = os.path.join(NTV_FLD, NTV_pdb)
NTV_pdb_dFile = os.path.join(DCK_FLD, NTV_pdb)

shutil.copy(NTV_pdb_nFile, NTV_pdb_dFile)
print(f'+ {NTV_pdb} > DOCKING folder')

---
---
 **04 | Preparing the Ligand**

Now, we start to prepare our ligands of interest by uploading the compound library in **`csv`** format onto the **`LIGAND`** folder. We will be using their SMILES notations (canonical/Isomeric) to generate the 3D energetically minimised ligand **`sd`** files based on specifed force field and conjugate gradient algorithm. We also parameterise our ligands to generate the **`pdbqt`** file required for virtual screening.

In [ ]:
# @title **Provide compound library**
# @markdown Upload a **< Compound Library >**  onto LIGAND folder.\
# @markdown The file **must** contain the below information with headers:
# @markdown + **`ID`** : The identity assigned to ligand
# @markdown + **`SMILES`** : The SMILES notation

Compoud_library = 'LIB' # @param {type:"string"}

CMPLIB_csv = Compoud_library + '.csv'
CMPLIB_csv_lFile = os.path.join(LIG_FLD, CMPLIB_csv)
CMPLIB_df = pd.read_csv(CMPLIB_csv_lFile)[['ID', 'SMILES']]
CMPLIB_df

In [ ]:
# @title **Generate ligand SD files**
# @markdown Select **< Gradient >** to set energy minimisation approach. **[ Default is conjugate ]** \
# @markdown Select **< Force Field >** to set MM method for energy minimisation. **[ Default is UFF ]** \
# @markdown Select **< Convergence Criteria >** to set minimum iteration difference required before stopping. **[ Default is 0.001 ]** \
# @markdown Select **< Maximum Steps >** to set maximum iterations allowed for minimisation. **[ Default is 10000 ]**

Gradient = 'conjugate' # @param ['steepest', 'conjugate']
Force_field = 'UFF' # @param ['GAFF', 'Ghemical', 'MMFF94', 'MMFF94s', 'UFF']
Convergence_criteria = '0.00001' # @param ['0.1', '0.01', '0.001', '0.0001', '0.00001', '0.000001', '0.0000001']
Maximum_steps = 10000 # @param {type: 'slider', min:1000, max:100000, step:1000}

def check_convergence(log_file: str) -> None:
    with open(log_file, 'r') as inpt:
        data = inpt.read()

    mol_data = [line.strip().split('\n') for line in data.split('#######')][:-1]
    uncon_id = [line[0][8:] for line in mol_data[1:] if 'CONVERGED' not in line[-2]]
    gradient = mol_data[0][0][10:]
    num_convg = len(mol_data[1:]) - len(uncon_id)
    uncon_str = 'EXCEPT ' + ', '.join(uncon_id) if uncon_id else ''

    print(f'+ Total {num_convg} ligand has achieved convergence with '
          f'{gradient} gradient {uncon_str}')

CMPLIB_obmin_log = Compoud_library + '_obmin.log'
CMPLIB_obmin_log_lfile = os.path.join(LIG_FLD, CMPLIB_obmin_log)
os.remove(CMPLIB_obmin_log_lfile) if os.path.exists(CMPLIB_obmin_log_lfile) else None

# Append 'GRADIENT' used for energy minimisation onto obmin.log
with open(CMPLIB_obmin_log_lfile, 'a') as log:
    log.write(f'GRADIENT: {Gradient}\n')

count = 0
statement = True
for LIG_ID, LIG_SMI in zip(tqdm(CMPLIB_df['ID']), CMPLIB_df['SMILES']):

    if statement:
        print(f'+ Selected {Force_field} '
              f'with {Gradient} gradient for energy minimisation '
              f'up to {Convergence_criteria} iteration difference or '
              f'at most {Maximum_steps:,} steps')
    statement = False

    LIG_sdf = LIG_ID + '.sdf'
    LIG_obmin_log = LIG_ID + '_obmin.log'
    LIG_sdf_lFile = os.path.join(LIG_FLD, LIG_sdf)

    # Append 'LIGAND ID' and 'DELIMITER' onto obmin.log
    with open(CMPLIB_obmin_log_lfile, 'a') as log:
        log.write(f'#######\n')
        log.write(f'LIGAND: {LIG_ID}\n')

    !obabel -:{'"'+LIG_SMI+'"'} -O {LIG_sdf_lFile} --title {LIG_ID} --gen3d \
    --best --minimize --ff {Force_field} --steps {Maximum_steps} \
    --crit {Convergence_criteria} --log &>> {CMPLIB_obmin_log_lfile}

    count += 1

# Append last 'DELIMITER' onto obmin.log
with open(CMPLIB_obmin_log_lfile, 'a') as log:
    log.write(f'#######')

check_convergence(CMPLIB_obmin_log_lfile)
print(f'+ {count} ligand.sdf > LIGAND folder')

In [ ]:
# @title **Visualise 3D ligand**
# @markdown Enter the **< Ligand >** to be viewed.

View = 'R002' # @param {type:'string'}
Representation_type = 'stick' # @param ['line', 'sphere', 'stick']
Show_atom_labels = False # @param {type:'boolean'}

LIG_view_lFile = os.path.join(LIG_FLD, View + '.sdf')

LBSP = LaboSpace(960, 640)
LBSP.load_ligand(LIG_view_lFile)\
    .set_style(
        show_represent=True,
        represent_type=Representation_type,
        represent_style={'colorScheme': 'lightGreyCarbon'})\
    .label_atoms(
        show_label=Show_atom_labels)
LBSP.view_space()

In [ ]:
# @title **Parameterise ligands with Gasteiger charges**
# @markdown This generate **`ligand.pdbqt`** files after:
# @markdown + Addition of Gasteiger Partial Charge
# @markdown + Removal of non-polar hydrogens

def pdbqt_add_chid(inpt_file: str) -> None:
    with open(inpt_file, 'r') as inpt:
        data = inpt.read()
    new_data = data.replace('  UNL  ', '  UNL A')
    with open(inpt_file, 'w') as oupt:
        oupt.write(new_data)

count = 0
for LIG_ID in tqdm(CMPLIB_df['ID']):
    LIG_sdf = LIG_ID + '.sdf'
    LIG_pdbqt = LIG_ID + '.pdbqt'
    LIG_dFLD = os.path.join(DCK_FLD, LIG_ID)
    LIG_sdf_lFile = os.path.join(LIG_FLD, LIG_sdf)
    LIG_pdbqt_dFFile = os.path.join(LIG_dFLD, LIG_pdbqt)

    try:
        os.makedirs(LIG_dFLD, exist_ok=True)
        !mk_prepare_ligand.py -i {LIG_sdf_lFile} -o {LIG_pdbqt_dFFile} > /dev/null 2>&1
        pdbqt_add_chid(LIG_pdbqt_dFFile)
        count += 1
    except:
        print(f'+ Error in preparing {LIG_ID}')
        os.rmdir(LIG_dFLD)
        continue

print(f'+ Parameterisation: {count} ligand.sdf > {count} ligand.pdbqt')
print(f'+ {count} ligand.pdbqt files > DOCKING folder')

---
---
 **05 | Setting Up Grid Box**

Here, we define a chemical search space with the use of grid box, which often centered within the binding, active or allosteric site of the target protein.

Built-in methods for defining grid box include:

+ **`LaBOX`** : Use the mean of XYZ extreme values from native ligand. [Link](https://github.com/RyanZR/LaBOX)
+ **`eBoxSize`** : Use the radius of gyration of native ligand. [Link](https://github.com/michal-brylinski/eboxsize)
+ **`eBoxSize-Mod`** : Similar to eBoxSize, but box center is computed using LaBOX method.
+ **`Autodock-Grid`** : 22.5 × 22.5 × 22.5 Å
+ **`Defined-by-Res`** : Use when important residues involved in binding interaction are known.
+ **`Blind-Docking`** : Use when binding pocket is not known.
+ **`Manual-Mode`** : Use the sliders below to define grid box.

In [ ]:
# @title **Place grid box at binding site**
# @markdown Select **< Method >** to set grid box at the center of binding site.

Residue_number = '' # @param {type: 'string'}
Method = 'LaBOX' # @param ['LaBOX', 'eBoxSize', 'eBoxSize-Mod', 'Autodock-Grid', 'Defined-by-Res', 'Manual-Mode']
Show_residues = True # @param {type: 'boolean'}

# @markdown **Manual Mode**

X = 0 # @param {type: 'number'}
Y = 0 # @param {type: 'number'}
Z = 0 # @param {type: 'number'}
Width = 10 # @param {type: 'number'}
Height = 10 # @param {type: 'number'}
Depth = 10 # @param {type: 'number'}
Scale = 1.0 # @param {type: 'number'}

LBSP = LaboSpace(960, 640)
LBSP_view = True

try:
    if Method == 'Defined-by-Res':
        GB = GridBox(PROT_pdb_dFile)
        center, bxsize = GB.defined_by_res(Residue_number)
    if Method == 'Manual-Mode':
        center = (X, Y, Z)
        bxsize = (round(Width, 3),
                  round(Height, 3),
                  round(Depth, 3))
except Exception as excp:
    print(f'+ {excp}')
    print(f'+ Error can be:\n'
            + '  - Expected \'Residue_number\'; found nothing\n'
            + '  - Expected \'protein PDB file\'; found nothing\n'
            + '  - Due to atomic error in \'protein PDB file\'')
    LBSP_view = False
else:
    LBSP.load_receptor(PROT_pdb_dFile)\
        .set_style(
            show_represent=True,
            represent_type='cartoon',
            represent_style={'color': 'white'})\
        .add_residues(
            show_residues=Show_residues,
            residue_number=Residue_number)

try:
    if Method == 'LaBOX':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.labox()
    if Method == 'eBoxSize':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.eboxsize()
    if Method == 'eBoxSize-Mod':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.eboxsize(modified=True)
    if Method == 'Autodock-Grid':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.autodock_grid()
except Exception as excp:
    print(f'+ {excp}')
    print(f'+ Expected \'native ligand PDB file\'; found nothing')
    print(f'+ Upload file to \'{NTV_FLD}\' and run \'Select native ligand\' cell to load it.')
    LBSP_view = False
else:
    try:
        LBSP.load_ligand(NTV_pdb_nFile)\
            .set_style(
                show_represent=True,
                represent_type='stick',
                represent_style={'colorScheme': 'greyCarbon'})
    except:
        pass

if LBSP_view:
    LBSP.add_gridbox(show_gridbox=True, center=center, bxsize=bxsize)
    LBSP.view_space()

In [ ]:
# @title **Generate docking config file**
# @markdown This generates **`config_file`** for AutoDock Vina.

cfg_name = 'config_file'
cfg_dFile = os.path.join(DCK_FLD, cfg_name)

with open(cfg_dFile, 'w') as cfg:
    cfg.write(f'center_x = {center[0]}\n')
    cfg.write(f'center_y = {center[1]}\n')
    cfg.write(f'center_z = {center[2]}\n')
    cfg.write(f'\n')
    cfg.write(f'size_x = {bxsize[0]}\n')
    cfg.write(f'size_y = {bxsize[1]}\n')
    cfg.write(f'size_z = {bxsize[2]}\n')

print(f'+ {cfg_name} > DOCKING folder')

---
---
 **06 | Performing Virtual Screening**

We now proceed with virtual screening using Autodock Vina for our target ligands in the compound library. The duration can vary, mainly influenced by factors including **number of ligands**, **number of rotatable bonds** and **level of exhaustiveness** selected. Do be patient as the process may take a few hours to complete.

In [ ]:
# @title **Run AutoDock Vina**
# @markdown Select **< Scoring >**  to set scoring option. **[ Default is vina ]** \
# @markdown Select **< Exhaustiveness >** to change the computational effort. **[ Default is 16 ]** \
# @markdown This generates a **`output.pdbqt`** containing different binding
# @markdown poses predicted for each docked ligand.

Scoring = 'vina' # @param ['vina', 'vinardo']
Exhaustiveness = '16' # @param [4, 8, 16, 32, 64, 128, 256]

cpu_cores = os.cpu_count()
LIG_dFLDs = sorted(folder[:-1] for folder in glob.glob(DCK_FLD + '/*/'))

# -- Start screening --
start = time.time()
count = 0
for folder in tqdm(LIG_dFLDs):
    LIG_ID = os.path.basename(folder)
    LIG_pdbqt = LIG_ID + '.pdbqt'
    LIG_oupt_log = LIG_ID + '_output.log'
    LIG_oupt_pdbqt = LIG_ID + '_output.pdbqt'
    LIG_dFLD = os.path.join(DCK_FLD, LIG_ID)
    LIG_pdbqt_dFFile = os.path.join(LIG_dFLD, LIG_pdbqt)
    LIG_oupt_log_dFFile = os.path.join(LIG_dFLD, LIG_oupt_log)
    LIG_oupt_pdbqt_dFFile = os.path.join(LIG_dFLD, LIG_oupt_pdbqt)

    %vina --receptor {PROT_pdbqt_dFile} --ligand {LIG_pdbqt_dFFile} \
    --out {LIG_oupt_pdbqt_dFFile} --config {cfg_dFile} --scoring {Scoring} \
    --exhaustiveness {Exhaustiveness} --cpu {cpu_cores} \
    --verbosity 2 &> {LIG_oupt_log_dFFile}

    count += 1
end = time.time()
# -- End screening --

print(f'')
print(f'+ Screening completed')
print(f'+ Time elapsed: ' + time.strftime('%Mm %Ss', time.gmtime(end - start)))
print(f'+ {count} ligand_output.pdbqt > DOCKING folder')

In [ ]:
# @title **Process output file**
# @markdown This generates **`ligand_[n].pdb`** for the binding poses of
# @markdown each docked ligand.

count = 0
for folder in tqdm(LIG_dFLDs):
    LIG_ID = os.path.basename(folder)
    LID_dash_pdb = LIG_ID + '_.pdb'
    LIG_oupt_pdbqt = LIG_ID + '_output.pdbqt'
    LIG_dash_pdb_dFFile = os.path.join(folder, LID_dash_pdb)
    LIG_oupt_pdbqt_dFFile = os.path.join(folder, LIG_oupt_pdbqt)

    with open(LIG_oupt_pdbqt_dFFile, 'r') as oupt:
        data = oupt.read()
        pose = data.count('MODEL')
        count += pose

    !obabel {LIG_oupt_pdbqt_dFFile} -O {LIG_dash_pdb_dFFile} -m > /dev/null 2>&1

print(f'+ Total {count} ligand_[n].pdb > DOCKING folder')

---
---
 **07 | Profiling Binding Interactions**

After docking, we use PLIP to determine the non-covalent binding interactions between the docked ligands and target protein. We will prepare the required complex **`pdb`** file containing docked ligand pose and target protein for PLIP to profile interactions.

In [ ]:
# @title **Generate complex PDB file**
# @markdown This merges the target protein with the binding poses of each docked
# @markdown ligand to produce individual **`cmpx.pdb`** file.

def generate_cmpx_pdb(inpt_prot: str, inpt_pose: str, oupt_cmpx: str) -> None:

    def write_line(line: str, keywords: list, oupt_file: str) -> None:
        header = line.split()[0]
        if header in keywords:
            oupt_file.write(line)

    def cmpx_writer() -> None:
        with open(oupt_cmpx, 'w') as oupt_file, \
             open(inpt_prot, 'r') as prot_file, \
             open(inpt_pose, 'r') as pose_file:
            for prot_line in prot_file:
                write_line(prot_line, prot_headers, oupt_file)
            for pose_line in pose_file:
                write_line(pose_line, pose_headers, oupt_file)

    prot_headers = ['ATOM', 'CONECT', 'TER']
    pose_headers = ['ATOM', 'CONECT', 'END']
    cmpx_writer()

count = 0
for folder in tqdm(LIG_dFLDs):
    LIG_ID = os.path.basename(folder)
    LIG_iFLD = os.path.join(INT_FLD, LIG_ID)
    os.makedirs(LIG_iFLD, exist_ok=True)

    LIG_pdb_dFFiles = sorted(glob.glob(folder + '/*.pdb'))
    for pose_file in LIG_pdb_dFFiles:
        pose_name = os.path.basename(pose_file).split('.')[0]
        cmpx_pdb = pose_name + '_cmpx.pdb'
        cmpx_pdb_ifile = os.path.join(LIG_iFLD, cmpx_pdb)
        generate_cmpx_pdb(PROT_pdb_dFile, pose_file, cmpx_pdb_ifile)
        count += 1

print(f'+ {count} ligand_[n]_cmpx.pdb > INTERACTION folder')

In [ ]:
# @title **Run PLIP**
# @markdown This generates an **`interpt.csv`** containing all detected binding
# @markdown interactions for each individual **`cmpx.pdb`** file. \
# @markdown Supported: **`HYDROPHOBIC`**, **`HBOND`**, **`WATERBRIDGE`**,
# @markdown **`SALTBRIDGE`**, **`PISTACKING`**, **`PICATION`**, **`HALOGEN`**, **`METAL`**. \

def interaction_profiler(inpt_cmpx: str, oupt_csv: str) -> None:

    int_bonds = ['HYDROPHOBIC', 'HBOND', 'WATERBRIDGE', 'SALTBRIDGE',
                 'PISTACKING', 'PICATION', 'HALOGEN', 'METAL']

    def BSR(inpt_cmpx: str) -> object:
        cmpx_mol = PDBComplex()
        cmpx_mol.load_pdb(inpt_cmpx)
        cmpx_lig = [lig for lig in cmpx_mol.ligands if lig.hetid == 'UNL'][0]
        cmpx_mol.characterize_complex(cmpx_lig)
        cmpx_int = cmpx_mol.interaction_sets['UNL:A:1']
        cmpx_rpt = BindingSiteReport(cmpx_int)
        return cmpx_rpt

    def BSR_feat(bsr: object) -> list:
        return [list(getattr(bsr, bond.lower() + '_features')) + ['BOND']
                for bond in int_bonds]

    def BSR_info(bsr: object) -> list:
        return [[list(info) + [bond] for info in getattr(bsr, bond.lower() + '_info')]
                for bond in int_bonds]

    def replace_column(df: pd.DataFrame, col_A: str, cols: list) -> pd.DataFrame:
        for col in cols:
            if col in df.columns:
                df[col_A] = df[col_A].fillna(df[col])
            else:
                pass
        return df

    def BSR_dataframe(bsr_feat: list, bsr_info: list) -> pd.DataFrame:
        bsr_df = []
        for feat, info in zip(bsr_feat, bsr_info):
            if info:
                df = pd.DataFrame(info, columns=feat)
            else:
                df = pd.DataFrame(columns=bsr_feat[0])
            bsr_df.append(df)
        BSR_df = pd.concat(bsr_df, ignore_index=True)
        BSR_df = replace_column(BSR_df, 'DIST', ['DIST_D-A', 'CENTDIST'])
        return BSR_df

    cmpx_bsr = BSR(inpt_cmpx)
    bsr_feat = BSR_feat(cmpx_bsr)
    bsr_info = BSR_info(cmpx_bsr)
    bsr_data = BSR_dataframe(bsr_feat, bsr_info)
    bsr_data.to_csv(oupt_csv, index=False)

LIG_iFLDs = sorted(folder[:-1] for folder in glob.glob(INT_FLD + '/*/'))

count = 0
for folder in tqdm(LIG_iFLDs):
    cmpx_pdb_iFFiles = sorted(glob.glob(folder + '/*_cmpx.pdb'))

    for cmpx_file in cmpx_pdb_iFFiles:
        cmpx_name = os.path.basename(cmpx_file).split('.')[0][:-5]
        inter_csv = cmpx_name + '_interpt.csv'
        inter_csv_iFile = os.path.join(folder, inter_csv)
        interaction_profiler(cmpx_file, inter_csv_iFile)
        count += 1

print(f'+ {count} ligand_[n]_interpt.pdb > INTERACTION folder')

---
---
 **08 | Analysing Screening Results**

We process all the docking and binding internaction results generated into for further evaluation. We will compute the RMSD between native ligand and docked ligands based on maximum common substructure (MCS). We will also visualise the docking pose in three-dimensional chemical space.

In [ ]:
# @title **Cluster and generate docking reports**
# @markdown Select **< Mode >** to specify clustering criteria.\
# @markdown This computes the LABO-RMSD and clusters docking results based on
# @markdown user-specified mode.\
# @markdown This also ranks the binding interaction based on their occurences from
# @markdown the clustered ligands. \
# @markdown Multiple **`report.csv`** file  will be generated including:
# @markdown + **`[ligand]_dockrpt.csv`** : Individual docking result of each ligand pose
# @markdown + **`[CMPLIB]_clusrpt.csv`** : Clustered docking results based on score and RMSD
# @markdown + **`[CMPLIB]_interpt.csv`** : Binding interactions result with bonds ranked by frequency

Mode = 'LABO-RMSD' # @param ['Best-Pose', 'LABO-RMSD']

def vina_report(inpt_file: str) -> dict:

    def vn_scores(inpt_file: str) -> float:
        with open(inpt_file, 'r') as inpt:
            for line in inpt:
                if 'REMARK VINA RESULT' in line:
                    dock_sc = float(line.split()[3])
                    rmsd_lb = float(line.split()[4])
                    rmsd_ub = float(line.split()[5])
        return dock_sc, rmsd_lb, rmsd_ub

    def vn_report(name: str, scores: tuple) -> dict:
        return {'NAME': [name], 'DOCK_SC': [scores[0]],
                'RMSD_LB': [scores[1]], 'RMSD_UB': [scores[2]]}

    inpt_name = os.path.basename(inpt_file).split('.')[0]
    scores = vn_scores(inpt_file)
    report = vn_report(inpt_name, scores)
    return report

def clustering(mode: str, rpt_df: pd.DataFrame) -> pd.DataFrame:

    def best_pose(df: pd.DataFrame) -> pd.DataFrame:
        df = df[rpt_df['NAME'].str.endswith('_1')]
        df = df.sort_values(['DOCK_SC'])
        return df

    def labo_rmsd(df: pd.DataFrame) -> pd.DataFrame:
        df.insert(0, 'LIG', df['NAME'].str[:-2])
        df = df.sort_values(['LIG', 'LABO_RMSD']).groupby(['LIG']).first()
        df = df.sort_values(['LABO_RMSD'])
        return df

    rpt_df = best_pose(rpt_df) if mode == 'Best-Pose' else labo_rmsd(rpt_df)
    rpt_df = rpt_df.drop(['RMSD_LB', 'RMSD_UB'], axis=1)
    rpt_df = rpt_df.reset_index(drop=True)
    return rpt_df

# Generate docking report for all ligand pose and MCS image
count = 0
for folder in tqdm(LIG_dFLDs):
    LIG_ID = os.path.basename(folder)
    LIG_MCS_png = LIG_ID + '_MCS.png'
    LIG_dockrpt_csv = LIG_ID + '_dockrpt.csv'
    LIG_MCS_png_dFFile = os.path.join(folder, LIG_MCS_png)
    LIG_dockrpt_csv_dFFile = os.path.join(folder, LIG_dockrpt_csv)
    LIG_pdb_dFFiles = sorted(glob.glob(folder + '/*.pdb'))

    RMSD = ComputeRMSD()
    dock_rpt = pd.DataFrame()

    try:
        ref_info = RMSD.load_molecule(NTV_pdb_dFile)

    except:
        print('+ Expected \'native ligand PDB file\' for RMSD calculation; found nothing.')
        print('+ Docking report generation proceeded without RMSD calculation.')
        for lig_file in LIG_pdb_dFFiles:
            vina_rpt = pd.DataFrame.from_dict(vina_report(lig_file))
            dock_rpt = pd.concat([dock_rpt, vina_rpt], ignore_index=True)

    else:
        for lig_file in LIG_pdb_dFFiles:
            lig_info = RMSD.load_molecule(lig_file)
            vina_rpt = pd.DataFrame.from_dict(vina_report(lig_file))
            rmsd_rpt = pd.DataFrame.from_dict(
                RMSD.rmsd_report(ref_info, lig_info, True, False, False)
                ).drop('NAME', axis=1)
            pose_rpt = pd.concat([vina_rpt, rmsd_rpt], axis=1)
            dock_rpt = pd.concat([dock_rpt, pose_rpt], ignore_index=True)
        RMSD.MCS_png.save(LIG_MCS_png_dFFile)

    finally:
        dock_rpt.to_csv(LIG_dockrpt_csv_dFFile, index=False)
        count += 1

print(f'+ {count} ligand_dockrpt.csv > DOCKING folder')

# Generate an overall docking report
CMPLIB_clusrpt_csv = Compoud_library + '_clusrpt.csv'
CMPLIB_clusrpt_csv_dFile = os.path.join(DCK_FLD, CMPLIB_clusrpt_csv)
CMPLIB_clus_rpt = pd.DataFrame()

for folder in tqdm(LIG_dFLDs):
    LIG_ID = os.path.basename(folder)
    LIG_dockrpt_csv = LIG_ID + '_dockrpt.csv'
    LIG_dockrpt_csv_dFFile = os.path.join(folder, LIG_dockrpt_csv)

    LIG_dock_rpt = pd.read_csv(LIG_dockrpt_csv_dFFile)
    CMPLIB_clus_rpt = pd.concat([CMPLIB_clus_rpt, LIG_dock_rpt], ignore_index=True)

CMPLIB_clus_rpt = clustering(Mode, CMPLIB_clus_rpt)
CMPLIB_clus_rpt.to_csv(CMPLIB_clusrpt_csv_dFile, index=False)

print(f'+ {Compoud_library}_clusrpt.csv > DOCKING folder')

# Generate a binding interaction report based on bond occurrences
CMPLIB_interpt_csv = Compoud_library + '_interpt.csv'
CMPLIB_interpt_csv_iFile = os.path.join(INT_FLD, CMPLIB_interpt_csv)
CMPLIB_inter_rpt = pd.DataFrame()

for CLIG_ID in tqdm(CMPLIB_clus_rpt['NAME']):
    CLIG_inter_csv = CLIG_ID + '_interpt.csv'
    folder = os.path.join(INT_FLD, CLIG_ID[:-2])
    CLIG_inter_csv_iFFile = os.path.join(folder, CLIG_inter_csv)

    CLIG_inter_rpt = pd.read_csv(CLIG_inter_csv_iFFile)
    residue_column = CLIG_inter_rpt['RESTYPE'] + ' ' + CLIG_inter_rpt['RESNR'].astype(str)
    CLIG_inter_rpt.insert(0, 'RESIDUE', residue_column)
    inter_rpt = CLIG_inter_rpt[['RESIDUE', 'BOND']]
    CMPLIB_inter_rpt = pd.concat([CMPLIB_inter_rpt, inter_rpt], ignore_index=True)

CMPLIB_inter_rpt = CMPLIB_inter_rpt.groupby(['RESIDUE', 'BOND']).agg(FREQ=('BOND', 'count'))
CMPLIB_inter_rpt = CMPLIB_inter_rpt.sort_values('FREQ')
CMPLIB_inter_rpt = CMPLIB_inter_rpt.reset_index()
CMPLIB_inter_rpt.to_csv(CMPLIB_interpt_csv_iFile, index=False)

print(f'+ {Compoud_library}_interpt.csv > INTERACTION folder')

In [ ]:
# @title **Rank docking result**
# @markdown This ranks the docking results for each best pose based on specifed input.
# @markdown + **`DOCK_SC`** : Autodock Vina Docking Score
# @markdown + **`LABO_RMSD`** : RMSD by Common Substructure (vs Native Ligand)

Sort_by = 'LABO_RMSD' # @param ['NAME', 'DOCK_SC', 'LABO_RMSD']
Order = 'ascending' # @param ['ascending', 'descending']
Top = 10 # @param {type: 'integer'}
Maximum_RMSD = 2.5 # @param {type:"number"}

order = {'ascending': True, 'descending': False}

CMPLIB_clusrpt_df = pd.read_csv(CMPLIB_clusrpt_csv_dFile)
CMPLIB_clusrpt_df = CMPLIB_clusrpt_df.sort_values([Sort_by], ascending=order[Order])
CMPLIB_clusrpt_df = CMPLIB_clusrpt_df[:Top]
CMPLIB_clusrpt_df = CMPLIB_clusrpt_df[CMPLIB_clusrpt_df['LABO_RMSD'] <= Maximum_RMSD]
CMPLIB_clusrpt_df = CMPLIB_clusrpt_df.reset_index(drop=True)
CMPLIB_clusrpt_df

In [ ]:
# @title **Show individual docking report**
# @markdown Enter the **< Ligand Name >** to view docking score and RMSD.
# @markdown + **`DOCK_SC`** : Autodock Vina Docking Score
# @markdown + **`RMSD_LB`** : RMSD Lower Bound (vs Best Pose)
# @markdown + **`RMSD_UB`** : RMSD Upper Bound (vs Best Pose)
# @markdown + **`LABO_RMSD`** : RMSD by Common Substructure (vs Native Ligand)

Ligand_name = 'R004' # @param {type : 'string'}

LIG_dFLD = os.path.join(DCK_FLD, Ligand_name)
LIG_dockrpt_csv = Ligand_name + '_dockrpt.csv'
LIG_dockrpt_csv_dFile = os.path.join(LIG_dFLD, LIG_dockrpt_csv)
LIG_dockrpt_df = pd.read_csv(LIG_dockrpt_csv_dFile)
LIG_dockrpt_df

In [ ]:
# @title **Rank binding interaction**
# @markdown This ranks the binding interactions for each best pose based on specifed input.
# @markdown + **`RESIDUE`** : Amino acids of protein
# @markdown + **`BOND`** : Type of binding interactions
# @markdown + **`FREQ`** : Frequency of occurrence

Sort_by = 'FREQ' # @param ['RESIDUE', 'BOND', 'FREQ']
Order = 'descending' # @param ['ascending', 'descending']
Hide_hydrophobic = True # @param {type: 'boolean'}
Show_barchart = True # @param {type: 'boolean'}

order = {'ascending': True, 'descending': False}

CMPLIB_interpt_df = pd.read_csv(CMPLIB_interpt_csv_iFile)
CMPLIB_interpt_df = CMPLIB_interpt_df.sort_values([Sort_by], ascending=order[Order])
CMPLIB_interpt_df = CMPLIB_interpt_df[
    CMPLIB_interpt_df['BOND'] != 'HYDROPHOBIC'
    ] if Hide_hydrophobic else CMPLIB_interpt_df
CMPLIB_interpt_df = CMPLIB_interpt_df.reset_index(drop=True)
display(CMPLIB_interpt_df)

if Show_barchart:
    width = len(CMPLIB_interpt_df) * 0.8
    snsax = plt.subplots(figsize=(width, 5))[1]
    bar_chart = sns.barplot(
        data=CMPLIB_interpt_df,
        x='RESIDUE',
        y='FREQ',
        hue='BOND',
        ax=snsax)
    bar_chart = plt.legend(title='BOND', loc='upper right')
    bar_chart = plt.xlabel('RESIDUE', fontweight='bold')
    bar_chart = plt.ylabel('FREQ', fontweight='bold')

In [ ]:
# @title **Show individual interaction profile**
# @markdown Enter the **< Pose Name >** to view the binding interactions.

Pose_name = 'R004_8' # @param {type : 'string'}

LIG_ID = Pose_name[:-2]
LIG_iFLD = os.path.join(INT_FLD, LIG_ID)
LIG_interpt_csv = Pose_name + '_interpt.csv'
LIG_interpt_csv_iFile = os.path.join(LIG_iFLD, LIG_interpt_csv)
LIG_interpt_dict = interaction_dict(LIG_interpt_csv_iFile, usage='view')
LIG_interpt_df = pd.DataFrame.from_dict(LIG_interpt_dict)
LIG_interpt_df = LIG_interpt_df[['RESTYPE', 'RESNR', 'DIST', 'BOND', 'COLOR']]
LIG_interpt_df

In [ ]:
# @title **Visualise 3D docking pose**
# @markdown **PROTEIN MODEL** \
# @markdown Enter the **< Protein >** to be viewed.

View_protein = '7N3N_prot_A' # @param {type: 'string'}
Protein_type = 'cartoon' # @param ['cartoon', 'cross', 'line', 'sphere', 'stick']
Protein_style = 'white' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Protein_opacity = 1 # @param {type: 'slider', min:0, max:1, step:0.1}
Residue_number = '' # @param {type: 'string'}
Surface_type = 'SES' # @param ['VDW', 'SAS', 'SES', 'MS']
Surface_style = 'isoelectric points' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Surface_opacity = 1 # @param {type: 'slider', min:0, max:1, step:0.1}
Show_protein = False # @param {type: 'boolean'}
Show_residue = False # @param {type: 'boolean'}
Show_surface = True # @param {type: 'boolean'}
Show_gridbox = False # @param {type: 'boolean'}

# @markdown **LIGAND MODEL** \
# @markdown Enter the **< Ligand >** to be viewed.

View_native_ligand = 'FUN_A' # @param {type: 'string'}
Native_ligand_style = 'stick' # @param ['cross', 'line', 'sphere', 'stick']
Show_native_ligand = True # @param {type: 'boolean'}
View_docked_ligand = 'R004_5' # @param {type: 'string'}
Docked_ligand_style = 'stick' # @param ['cross', 'line', 'sphere', 'stick']
Show_docked_ligand = True # @param {type: 'boolean'}

# @markdown **INTERACTION MODEL** \
# @markdown Enter the **< Interaction Type >** to be viewed. \
# @markdown Select or combine from **`HYDROPHOBIC`**, **`HBOND`**, **`WATERBRIDGE`**,
# @markdown **`SALTBRIDGE`**, **`PISTACKING`**, **`PICATION`**, **`HALOGEN`**, **`METAL`**. \
# @markdown *Note: All interactions are selected if not provided.*

Interaction_type = "HBOND, WATERBRIDGE, SALTBRIDGE, PISTACKING, PICATION, HALOGEN, METAL" # @param {type: 'string'}
Show_interaction = True # @param {type: 'boolean'}

# @markdown **OTHER OPTIONS** \
# @markdown Miscellaneous visualisation settings.

Slab_view = False # @param {type: 'boolean'}

dLIG_dFLD = os.path.join(DCK_FLD, View_docked_ligand[:-2])
iLIG_iFLD = os.path.join(INT_FLD, View_docked_ligand[:-2])
PROT_view_dFile = os.path.join(DCK_FLD, View_protein + '.pdb')
NTV_view_dFile = os.path.join(DCK_FLD, View_native_ligand + '.pdb')
LIG_view_dFFile = os.path.join(dLIG_dFLD, View_docked_ligand + '.pdb')
INT_view_iFFile = os.path.join(iLIG_iFLD, View_docked_ligand + '_interpt.csv')
cfg_center, cfg_bxsize = extract_config(cfg_dFile)

LBSP = LaboSpace(960, 640)
LBSP.load_receptor(PROT_view_dFile)\
    .set_style(
        show_represent=Show_protein,
        represent_type=Protein_type,
        represent_style=builtin_style(
            style=Protein_style,
            opacity=Protein_opacity))\
    .add_residues(
        show_residues=Show_residue,
        residue_number=Residue_number)\
    .add_surface(
        show_surface=Show_surface,
        surface_type=Surface_type,
        surface_style=builtin_style(
            style=Surface_style,
            opacity=Surface_opacity))

try:
    LBSP.load_ligand(NTV_view_dFile)\
        .set_style(
            show_represent=Show_native_ligand,
            represent_type=Native_ligand_style,
            represent_style={'color': 'grey'})
except:
    pass

LBSP.load_ligand(LIG_view_dFFile)\
    .set_style(
        show_represent=Show_docked_ligand,
        represent_type=Docked_ligand_style,
        represent_style={'colorscheme': 'salmonCarbon'})
LBSP.add_interaction(
    interaction_file=INT_view_iFFile,
    show_interaction=Show_interaction,
    select_interaction=Interaction_type)
LBSP.add_gridbox(
    show_gridbox=Show_gridbox,
    center=cfg_center,
    bxsize=cfg_bxsize)
LBSP.view_space(
    slab_view=Slab_view)

show_cscale({Show_protein: Protein_style}, {Show_surface: Surface_style})

---
---
 **09 | Save to Google Drive**

Finally, we save all the generated data into Google Drive. A copy of the current working directory will be stored in our specified destination in Google Drive.

In [ ]:
# @title **Mount Google Drive**
# @markdown This flush and mount Google Drive.

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
# @title **Store data**
# @markdown Enter the **< Destination >** for storage. \
# @markdown This copies all generated data from current working directory into
# @markdown specified destination.

Destination = '/content/drive/MyDrive' # @param {type:'string'}

DST_FLD = os.path.join(Destination, Job_name)
shutil.copytree(WRK_DIR, DST_FLD)

print(f'+ {Job_name} stored in {DST_FLD}')

#**Phase 5 . Molecular dynamic Simulation.Gromacs**

GROMACS is a molecular dynamics package mainly designed for simulations of proteins, lipids, and nucleic acids. It was originally developed in the Biophysical Chemistry department of University of Groningen, and is now maintained by contributors in universities and research centers worldwide.


[![DOI](https://img.shields.io/badge/DOI-10.101002/zenodo.8246977-blue)](https://pubmed.ncbi.nlm.nih.gov/16211538/)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Let us check the Google COlab resources - 1GPU and 2 CPU with 1TB HDD and 12GB RAM
%%shell
lscpu
nvidia-smi

In [ ]:
%%shell
wget https://github.com/Kitware/CMake/releases/download/v3.20.0-rc1/cmake-3.20.0-rc1.tar.gz
tar xfz cmake-3.20.0-rc1.tar.gz

In [ ]:
%%shell
mkdir /content/cmake-3.20.0-rc1/build
cd /content/cmake-3.20.0-rc1/build
cmake /content/cmake-3.20.0-rc1/
make -j 2
make install

In [ ]:
%%shell
wget ftp://ftp.gromacs.org/gromacs/gromacs-2024.tar.gz
tar -xzvf gromacs-2024.tar.gz
cd /content/gromacs-2024
mkdir /content/gromacs-2024/build
cd /content/gromacs-2024/build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA
make -j 2
make install
source /usr/local/gromacs/bin/GMX

In [ ]:
%%shell
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/X1B341MD/content/14
ls

##gmx pdb2gmx -f "afx1b341v2.pdb" -o "afx1b341v2.gro" -p "afx1b341v2.top" -ff charmm27 -water tip3p -ignh -missing
##gmx editconf -f "afx1b341v2.gro" -c -d 1 -bt cubic -o
##gmx solvate -cp out.gro -cs -p "afx1b341v2.top" -o "afx1b341v2_box"
##gmx grompp -f ions.mdp -c "afx1b341v2_box.gro" -p "afx1b341v2.top" -o "afx1b341v2_charged" -maxwarn 2
##echo 'SOL' | gmx genion -s "afx1b341v2_charged.tpr" -o "afx1b341v2_charged" -p "afx1b341v2.top" -neutral
##grace -nxy "afx1b341v2_potentialsd.xvg" -hdevice PNG -hardcopy -printfile "../content/drive/X1B341/MyDrive/X1B341MD/content/14/fig/afx1b341v2_potentialsd.png"

##gmx grompp -f PME_em.mdp -c "afx1b341v2_charged.gro" -p "afx1b341v2.top" -o "afx1b341v2_charged" -maxwarn 2
##gmx mdrun -v -s "afx1b341v2_charged.tpr" -deffnm "afx1b341v2_sd_em"
##echo '10 0' | gmx energy -f "afx1b341v2_sd_em.edr" -o "afx1b341v2_potentialsd.xvg"
##grace -nxy "afx1b341v2_potentialsd.xvg" -hdevice PNG -hardcopy -printfile
##apt-get install grace

##gmx grompp -f PME_emcg.mdp -c "afx1b341v2_sd_em.gro" -p "afx1b341v2.top" -o "afx1b341v2_cg_em" -maxwarn 2
##gmx mdrun -v -s "afx1b341v2_cg_em.tpr" -deffnm "afx1b341v2_cg_em"
##echo '10 0' | gmx energy -f "afx1b341v2_cg_em.edr" -o "afx1b341v2_potentialcg.xvg"
##grace -nxy "afx1b341v2_potentialcg.xvg" -hdevice PNG -hardcopy -printfile "/content/content/fig/afx1b341v2_potentialcg.png"
##gmx grompp -f nvt4.2.mdp -c "afx1b341v2_cg_em.gro" -r "afx1b341v2_cg_em.gro" -p "afx1b341v2.top" -o "afx1b341v2_nvt.tpr" -maxwarn 2
##gmx mdrun -v -s "afx1b341v2_nvt.tpr" -deffnm "afx1b341v2_nvt"
##echo '16 0' | gmx energy -f "afx1b341v2_nvt.edr" -o "afx1b341v2_temperature_nvt.xvg"
##grace -nxy "afx1b341v2_temperature_nvt.xvg" -hdevice PNG -hardcopy -printfile "/content/content/fig/afx1b341v2_temperature_nvt.png"
##gmx grompp -f npt4.1.mdp -c "afx1b341v2_nvt.gro" -r "afx1b341v2_nvt.gro" -p "afx1b341v2.top" -o "afx1b341v2_npt.tpr" -maxwarn 2
##gmx mdrun  -v -s "afx1b341v2_npt.tpr" -deffnm "afx1b341v2_npt"
##echo '16 0' | gmx energy -f "afx1b341v2_npt.edr" -o "afx1b341v2_temperature_npt.xvg"
#çgmx grompp -f md5.mdp -c "afx1b341v2_npt.gro" -p "afx1b341v2.top" -o "afx1b341v2_pr" -maxwarn 2
gmx mdrun  -v -s "afx1b341v2_pr.tpr" -deffnm "afx1b341v2_pr"

##gmx grompp -f md5.mdp -p afx1b341v2.top -c afx1b341v2_npt.gro -t afx1b341v2_pr.cpt -o new.tpr
##gmx mdrun   -deffnm afx1b341v2_pr2  -cpi afx1b341v2_pr.cpt -s new.tpr -append
##gmx mdrun -v -s new.tpr -cpi afx1b341v2_pr.cpt -deffnm afx1b341v2_pr


# **Phase 6. Rapid protein stability prediction using deep learning representations**

Preprint pipeline version for predicting protein variants **thermodynamic stability changes** ($\Delta \Delta G$) using a deep learning representation. The program, using as input a protein structure (uploaded as PDB) returns stability predictions ($\Delta \Delta G$ in kcal/mol) for each variant at each position of the query protein.


[![DOI](https://img.shields.io/badge/DOI-10.1101/zenodo.8246977-blue)](https://www.biorxiv.org/content/10.1101/2022.07.14.500157v1)


##  <b><font color='#009e74'> Reminders and Important informations:</font></b>
- This notebook  <b><font color='#d55c00'>must</font></b> be run in a Colab GPU session (go to page menu: `Runtime`->  `Change runtime type` -> select `GPU` and confirm
- Run <b><font color='#d55c00'>ONE</font></b> cell at a time, <b><font color='#d55c00'>DON'T USE</font></b> the `Runtime`->  `Run all` function as condacolab installation requires a restart of the kernel.
- Cells labelled <b><font color='#56b4e9'>PRELIMINARY OPERATIONS </font></b>  must be run <b><font color='#d55c00'>ONE</font></b> at a time and <b><font color='#d55c00'>ONCE</font></b> at the start and skipped for new predictions.
- Cells named as  <b><font color='#56b4e9'>PRELIMINARY OPERATIONS </font></b> have to be run <b><font color='#d55c00'>ONE BY ONE</font></b>  and <font color='#d55c00'>ONCE only at the start</font></b>  and  skipped for new predictions.
- <b><font color='#d55c00'>ONE</font></b> single pdb at the time can be processed by the pipeline.
- A  <b><font color='#d55c00'>new run</font></b> can be perform input direcly the new structure in the pdb upload cell and run the prediction cell again

****

## <b><font color='#009e74'>PIPELINE : PRELIMINARY OPERATIONS </font></b>
These cells MUST be run <b><font color='#d55c00'>INDIVIDUALLY AND SEQUENTIALLY</font></b>, and only once at the start of the notebook.
****

In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Install condacolab
#@markdown Run this cell to install condacolab. After running this cell the kernel will be automatically restarted (wait ~1min before run the next cell)

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Setup enviroment and dependencies</b>

#@markdown Run this cell to install the required enviroment and dependencies (~10 minutes)

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**
! rm -r sample_data

# install dependencies present in pip
! pip install numpy==1.17.3 torch==1.7.1 biopython==1.72 matplotlib pdb-tools &> /dev/null
! pip install --upgrade pdb-tools
! echo "-> packages with pip installed!"

! mamba install  mpl-scatter-density ptitprince pdbfixer openmm=7.5.1 pandas=1.4.4 -c omnia -c conda-forge -c anaconda -c defaults --yes &> /dev/null
! echo "-> packages with conda installed!"



In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Retrieve parameters and RaSP files</b>

#@markdown Run this cell to import RaSP files and parameters

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**
%%bash

#install svn
apt-get install -qq subversion &> /dev/null

#mkdir of necessary folders
mkdir data
mkdir data/test
mkdir data/test/predictions
mkdir data/test/predictions/raw
mkdir data/test/predictions/cleaned
mkdir data/test/predictions/parsed
mkdir output/
mkdir output/predictions

#download project folders from github

svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/src  &> /dev/null
svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/output/cavity_models  &> /dev/null
svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/output/ds_models  &> /dev/null

wget -cq https://github.com/KULL-Centre/papers/raw/papers/2022/ML-ddG-Blaabjerg-et-al/data/pdb_frequencies.npz -o /content/data/pdb_frequencies.npz
wget -cq https://github.com/KULL-Centre/papers/raw/main/2022/ML-ddG-Blaabjerg-et-al/colab_additonals/colab_additional.zip

#extra files for runnin the notebooks

mv ds_models ./output/
mv cavity_models ./output/

unzip colab_additional.zip &> /dev/null
rm colab_additional.zip

mv /content/colab_additional/best_model_path.txt /content/output/cavity_models/
mv /content/colab_additional/clean_pdb.py /content/src/pdb_parser_scripts/
mv /content/colab_additional/helpers.py /content/src/
mv /content/colab_additional/pdb_frequencies.npz /content/data/

echo "---> Github data imported!"

#get and compile reduce

cd src/pdb_parser_scripts
git clone https://github.com/rlabduke/reduce.git
cd reduce/
make &> /dev/null

mv /content/colab_additional/reduce /content/src/pdb_parser_scripts/reduce/

chmod +x /content/src/pdb_parser_scripts/reduce/reduce
echo "----> reduce installed"

rm -r /content/colab_additional
#@markdown ****

In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Import python libraries, functions and setup common variables</b>

#@markdown Run this cell to import libraries and functions necessary for the pipeline.

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**

import sys
sys.path.append("/usr/local/lib/python3.7/site-packages")

import glob
import os
import random
import pathlib
import subprocess
import numpy as np
import pandas as pd
import torch
import time
import datetime
import matplotlib
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
from Bio.PDB.Polypeptide import index_to_one, one_to_index
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset
from google.colab import files

sys.path.append('./src/')

from cavity_model import (
     CavityModel,
     DownstreamModel,
     ResidueEnvironment,
     ResidueEnvironmentsDataset,
)

from helpers import (
     populate_dfs_with_resenvs,
     remove_disulfides,
     fermi_transform,
     inverse_fermi_transform,
     init_lin_weights,
     ds_pred,
     cavity_to_prism,
     get_seq_from_variant,
)

from visualization import (
     hist_plot,
)

#Extra function to fix pdb

# Setup pipeline parameters
## Set seeds
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Main deep parameters
DEVICE = "cuda"  # "cpu" or "cuda"
NUM_ENSEMBLE = 10
TASK_ID = int(1)
PER_TASK = int(1)

#@markdown ****

In [ ]:
#@title <b><font color='#56b4e9'> PDB upload</font></b>

#@markdown Choose between <b><font color='#d55c00'> ONE</font></b> of the possible input sources for the target pdb and <b><font color='#d55c00'>leave the other cells empty or unmarked</font></b>
#@markdown - AlphaFold2 PDB (v4) via Uniprot ID:
AF_ID ='P68871'#@param {type:"string"}
#@markdown - PDB ID (imported from RCSB PDB):
PDB_ID =''#@param {type:"string"}
#@markdown - Upload custom PDB
PDB_custom =False#@param {type:"boolean"}

#@markdown

#@markdown Select target chain (default A)
chain='A' #@param {type:'string'}

if os.path.exists("/content/query_protein.pdb"):
    os.remove("/content/query_protein.pdb")
if os.path.exists("/content/data/test/predictions/raw/query_protein_uniquechain.pdb"):
    os.remove("/content/data/test/predictions/raw/query_protein_uniquechain.pdb")
if os.path.exists("/content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb"):
    os.remove("/content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb")
if os.path.exists("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz"):
    os.remove("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz")

if PDB_custom:
  print('Upload PDB file:')
  uploaded_pdb = files.upload()
  for fn in uploaded_pdb.keys():
    os.rename(fn, f"/content/query_protein.pdb")
    print('PDB file correctly loaded')
elif (AF_ID !='') and (len(AF_ID)>=6) :
    subprocess.call(['curl','-s','-f',f'https://alphafold.ebi.ac.uk/files/AF-{AF_ID}-F1-model_v4.pdb','-o','/content/query_protein.pdb'])
elif (PDB_ID !='') and (len(PDB_ID)==4):
    subprocess.call(['curl','-s','-f',f'https://files.rcsb.org/download/{PDB_ID}.pdb','-o','/content/query_protein.pdb'])

else:
  print(f'ERROR: any PDB uploaded, please select one of the above inputs')

#@markdown N.B. This cell will also perform preliminary operations to correcly format the uploaded PDB

## remove other chains and move to raw folder
!pdb_selchain -"$chain" /content/query_protein.pdb | pdb_delhetatm | pdb_delres --999:0:1 | pdb_fixinsert | pdb_tidy  > /content/data/test/predictions/raw/query_protein_uniquechain.pdb
# Select PDBs to run during this task - could be simplified if we decide to set PER_TASK = 1 for all cases

pdb_input_dir = "data/test/predictions/raw/"
input_pdbs = sorted(list(filter(lambda x: x.endswith(".pdb"), os.listdir('data/test/predictions/raw/'))))
start = (TASK_ID-1)*(PER_TASK)
end = (TASK_ID*PER_TASK)
if end > len(input_pdbs):
    end = len(input_pdbs) #avoid end index exceeding length of list
pdbs = input_pdbs[start:end]
pdb_names = [i.split(".")[0] for i in pdbs]
print(pdb_names)
print(f"Pre-processing PDBs ...")

!python3 /content/src/pdb_parser_scripts/clean_pdb.py --pdb_file_in /content/data/test/predictions/raw/query_protein_uniquechain.pdb --out_dir /content/data/test/predictions/cleaned/ --reduce_exe /content/src/pdb_parser_scripts/reduce/reduce #&> /dev/null
!python3 /content/src/pdb_parser_scripts/extract_environments.py --pdb_in /content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb  --out_dir /content/data/test/predictions/parsed/  #&> /dev/null

if os.path.exists("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz"):
  print(f"Pre-processing PDBs correctly ended")
else:
  print(f"Pre-processing PDB didn't end correcly, please check input informations")

#@markdown ****

In [ ]:

#@title <b><font color='#56b4e9'> Pipeline RUN </font></b>

#@markdown <b><font color='#d55c00'>Execute the cell</font></b> to run the pipeline and generate **saturation mutagenesis predictions of thermodynamic stability changes** predictions

### Pre-process structure data

# Create temporary residue environment datasets to more easily match ddG data
pdb_filenames_ds = sorted(glob.glob(f"/content/data/test/predictions/parsed/*coord*"))

dataset_structure = ResidueEnvironmentsDataset(pdb_filenames_ds, transformer=None)

resenv_dataset = {}
for resenv in dataset_structure:
    if AF_ID!='':
      key = (f"--{AF_ID}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    elif PDB_ID!='':
      key = (f"--{PDB_ID}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    else:
      key = (f"--{'CUSTOM'}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    resenv_dataset[key] = resenv
df_structure_no_mt = pd.DataFrame.from_dict(resenv_dataset, orient='index', columns=["resenv"])
df_structure_no_mt.reset_index(inplace=True)
df_structure_no_mt["index"]=df_structure_no_mt["index"].astype(str)
res_info = pd.DataFrame(df_structure_no_mt["index"].str.split('--').tolist(),
                        columns = ['blank','pdb_id','chain_id','pos','wt_AA', 'blank2'])

df_structure_no_mt["pdbid"] = res_info['pdb_id']
df_structure_no_mt["chainid"] = res_info['chain_id']
df_structure_no_mt["variant"] = res_info["wt_AA"] + res_info['pos'] + "X"
aa_list = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L",
            "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
df_structure = pd.DataFrame(df_structure_no_mt.values.repeat(20, axis=0), columns=df_structure_no_mt.columns)
for i in range(0, len(df_structure), 20):
    for j in range(20):
        df_structure.iloc[i+j, :]["variant"] = df_structure.iloc[i+j, :]["variant"][:-1] + aa_list[j]
df_structure.drop(columns="index", inplace=True)

# Load PDB amino acid frequencies used to approximate unfolded states
pdb_nlfs = -np.log(np.load(f"{os.getcwd()}/data/pdb_frequencies.npz")["frequencies"])

# # Add wt and mt idxs and freqs to df

df_structure["wt_idx"] = df_structure.apply(lambda row: one_to_index(row["variant"][0]), axis=1)
df_structure["mt_idx"] = df_structure.apply(lambda row: one_to_index(row["variant"][-1]), axis=1)
df_structure["wt_nlf"] = df_structure.apply(lambda row: pdb_nlfs[row["wt_idx"]], axis=1)
df_structure["mt_nlf"] = df_structure.apply(lambda row: pdb_nlfs[row["mt_idx"]], axis=1)

# Define models
best_cavity_model_path = open(f"/content/output/cavity_models/best_model_path.txt", "r").read()
cavity_model_net = CavityModel(get_latent=True).to(DEVICE)
cavity_model_net.load_state_dict(torch.load(f"{best_cavity_model_path}"))
cavity_model_net.eval()
ds_model_net = DownstreamModel().to(DEVICE)
ds_model_net.apply(init_lin_weights)
ds_model_net.eval()

###set start time
start_time = time.perf_counter()

# Make ML predictions
print(f"Starting downstream model prediction")
dataset_key="predictions"
df_ml = ds_pred(cavity_model_net,
                ds_model_net,
                df_structure,
                dataset_key,
                NUM_ENSEMBLE,
                DEVICE,
                )
print(f"Finished downstream model prediction")
end_time = time.perf_counter()
elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Complete - prediction execution took", elapsed)

elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Generating output files")
#Merge and save data with predictions

df_total = df_structure.merge(df_ml, on=['pdbid','chainid','variant'], how='outer')
#df_total["b_factors"] = df_total.apply(lambda row: row["resenv"].b_factors, axis=1)
df_total = df_total.drop("resenv", 1)
print(f"{len(df_structure)-len(df_ml)} data points dropped when matching total data with ml predictions in: {dataset_key}.")

# Parse output into separate files by pdb, print to PRISM format
for pdbid in df_total["pdbid"].unique():
    df_pdb = df_total[df_total["pdbid"]==pdbid]
    for chainid in df_pdb["chainid"].unique():
        pred_outfile = f"{os.getcwd()}/output/{dataset_key}/cavity_pred_{pdbid}_{chainid}.csv"
        print(f"Parsing predictions from pdb: {pdbid}{chainid} into {pred_outfile}")
        df_chain = df_pdb[df_pdb["chainid"]==chainid]
        df_chain = df_chain.assign(pos = df_chain["variant"].str[1:-1])
        df_chain['pos'] = pd.to_numeric(df_chain['pos'])
        first_res_no = min(df_chain["pos"])
        df_chain = df_chain.assign(wt_AA = df_chain["variant"].str[0])
        df_chain = df_chain.assign(mt_AA = df_chain["variant"].str[-1])
        seq = get_seq_from_variant(df_chain)
        df_chain.to_csv(pred_outfile, index=False)
        prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{pdbid}_{chainid}.txt"

        # if (AF_ID !=''):
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{AF_ID}_{chainid}.txt"
        # elif (PDB_ID !=''):
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{PDB_ID}_{chainid}.txt"
        # elif PDB_custom:
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_XXXX_{chainid}.txt"
        cavity_to_prism(df_chain, pdbid, chainid, seq, prism_outfile)

# End timer and print result
#!rm /content/output/predictions/*xxxx*.csv
elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Complete - files generated")

#@markdown ****

In [ ]:
#@title <b><font color='#56b4e9'> Download results as archive </font></b>

#@markdown Run the cell to <b><font color='#009e74'> download a .zip archive </font></b> with prediction files for the <ins>current run</ins>.

#@markdown <ins>Tick</ins> the next box if you ran multiple predictions and you want to <ins>download all of them</ins>.

download_all_predictions= False #@param {type:"boolean"}

if download_all_predictions:
  os.system( "zip -r {} {}".format( f"predictions_output_all.zip" , f"/content/output/predictions/*" ) )
  files.download(f"predictions_output_all.zip")
else:
  if (AF_ID !=''):
    os.system( "zip -r {} {}".format( f"predictions_output_{AF_ID}.zip" , f"/content/output/predictions/*{AF_ID}*" ) )
    files.download(f"predictions_output_{AF_ID}.zip")
  elif (PDB_ID !=''):
    os.system( "zip -r {} {}".format( f"predictions_output_{PDB_ID}.zip" , f"/content/output/predictions/*{PDB_ID}*" ) )
    files.download(f"predictions_output_{PDB_ID}.zip")
  else:
    os.system( "zip -r {} {}".format( f"predictions_output.zip" , f"/content/output/predictions" ) )
    files.download(f"predictions_output.zip")

  if download_all_predictions:
    os.system( "zip -r {} {}".format( f"predictions_output.zip" , f"/content/output/predictions" ) )
    files.download(f"predictions_output_all.zip")

#@markdown **P.S.: prediction files are also stored in the colab file system folder: `/output/predictions/`**
#@markdown ****

**CHANGING LOG**

- 12th April 2023: AlphaFold2 database version update to v4.
- 3rd April 2023: Torch version and packagess updated for python 3.9



**Troubleshooting**

- Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
- Try to restart the session "Runtime" -> "Factory reset runtime".
- Run the PRELIMINARY OPERATION one at the time, to avoid crashes.
- Check your input pdb.

\\

**Known problems:**

- Condacolab need to restart the notebook kernel, so preliminart cells MUST be run one at the time to allow this.
- Residues with numeration index below 0 are not supported by the output file parser and thus they deleted from the pdb in the pre-processing step.
- Insertion annotations in the pdb are not supported. Any annotations is actually deleted during the pre-processing step.




#**Phase 8 . Carcinogenicity Prediction**


Metabokiller offers a novel, machine learning-based approach that accurately recognizes carcinogens by quantitatively assessing their chemical composition as well as potential to induce proliferation, oxidative stress, genomic instability, alterations in epigenetic signatures, and activation of anti-apoptotic pathways, and therefore, obviates the absolute need for bonafide (non)carcinogens for training model. Concomitant with the carcinogenicity prediction, it also reveals the contribution of the aforementioned biochemical processes in carcinogenicity, thereby making the proposed approach highly interpretable.



[![DOI](https://img.shields.io/badge/DOI-10.10138/zenodo.8246977-blue)](https://www.nature.com/articles/d44151-022-00084-8/)



In [ ]:
%%shell
pip install signaturizer
pip install lime
pip install kora -q
pip install Metabokiller==0.1

In [ ]:

import kora.install.rdkit
from Metabokiller import mk_predictor as mk
from Metabokiller import EnsembleMK

In [ ]:
smiles = ['OC(=O)CCC(C(=O)O)O']
result = mk.Epigenetics(smiles)
#Output predction dataframe
result

**Meatbokiller Ensemble with Molecular Explanation**

In [ ]:
#Additional packages for Explanation Graphs
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import pyplot as plt

In [ ]:
smiles = ['OC(=O)CCC(C(=O)O)O']

In [ ]:
result,explanation = EnsembleMK.predict(['CC(C)[N+](=O)[O-]'],explainability=True)

In [ ]:
#Output predction dataframe
result

In [ ]:
#Saving the Molecular Explanation graphs into pdfs
pdf = PdfPages("Ensmble-Result.pdf")
for fig in explanation:
  fig.savefig(pdf, format='pdf')
pdf.close()